In [1]:
from __future__ import division
import numpy as np

def compute_const_coeffs(eta, gamma, a):

    # Define frequently-used constants
    asq = a*a
    pisq = np.pi*np.pi

    # Define constants that determine the fitting parameter K
    # See the discussion in https://arxiv.org/pdf/1311.2544.pdf between Equations (3) and (4)
    K0 = 1.712
    K1 = -1.803949138004582
    K2 = -39.77229225266885
    K3 = 103.16588921239249

    # Compute the fitting parameter K
    # See https://arxiv.org/abs/0912.3517 Equation (5.67) and the discussion following Equation 6.9
    # as well as https://arxiv.org/pdf/1311.2544.pdf
    K = K0 + K1*eta + K2*eta*eta + K3*eta*eta*eta

    # Define more frequently-used constants
    EtaKm1 = eta*K - 1.
    EtaKm1sq = EtaKm1*EtaKm1

    # Compute the Post-Newtonian coefficients
    # See https://arxiv.org/abs/0912.3517 Equations (5.77) to (5.81) and
    # https://arxiv.org/pdf/1311.2544.pdf Equation (2)
    Delta0 = K*(EtaKm1 - 1.)
    Delta1 = -2.*(Delta0 + K)*EtaKm1

    Delta1sq = Delta1*Delta1
    Delta1cu = Delta1*Delta1sq
    Delta1ft = Delta1cu*Delta1

    Delta2 = 0.5*Delta1*(Delta1 - 4.*EtaKm1) - asq*EtaKm1sq*Delta0

    Delta2sq = Delta2*Delta2

    Delta3 = -Delta1cu/3. + Delta1*Delta2 + Delta1sq*EtaKm1 - 2.*(Delta2 - EtaKm1)*EtaKm1 - asq*Delta1*EtaKm1sq
    Delta4 = 1./12.*(6*asq*(Delta1sq - 2*Delta2)*EtaKm1sq + 3*Delta1ft - 8*EtaKm1*Delta1cu - 12*Delta2*Delta1sq
                     + 12*(2*EtaKm1*Delta2 + Delta3)*Delta1 + 12*(94./3. - 41./32.*pisq)*EtaKm1sq
                     + 6*(Delta2*Delta2 - 4*Delta3*EtaKm1))
    Delta5 = EtaKm1sq*(-4237./60. + 128./5.*gamma + 2275./512.*pisq - asq*(Delta1cu - 3.*Delta1*Delta2 + 3.*Delta3)/3.
                     - (Delta1ft*Delta1 - 5.*Delta1cu*Delta2 + 5.*Delta1*Delta2sq + 5.*Delta1sq*Delta3
                        - 5.*Delta2*Delta3 - 5.*Delta1*Delta4)/(5.*EtaKm1sq) + (Delta1ft - 4.*Delta1sq*Delta2
                        + 2.*Delta2sq + 4.*Delta1*Delta3 - 4.*Delta4)/(2*EtaKm1) + (256./5.)*np.log(2))
    Delta5l = (64./5.)*EtaKm1sq

    #Add comment here
    dSO = -74.71 - 156.*eta + 627.5*eta*eta
    dSS = 8.127 - 154.2*eta + 830.8*eta*eta

    return K, Delta0, Delta1, Delta2, Delta3, Delta4, Delta5, Delta5l, dSO, dSS


## dx dpy

In [2]:
def compute_d2Hdxdp2(m1, m2, eta, x, y, z, p1, p2, p3, S1x, S1y, S1z, S2x, S2y, S2z, KK, k0, k1, dSO, dSS, tortoise, EMgamma):
    M = m1+m2
    mu = m1*m2/M
    eta = mu/M
    r = np.sqrt(x*x+y*y+z*z)
    u = 1/r
    sigmastar3 = m2/m1*S1z+m1/m2*S2z
    sigmastar2 = m2/m1*S1y+m1/m2*S2y
    sigmastar1 = m2/m1*S1x+m1/m2*S2x
    sigma3 = S1z+S2z
    sigma2 = S1y+S2y
    sigma1 = S1x+S2x
    Skerr3 = sigma3
    Skerr2 = sigma2
    Skerr1 = sigma1
    Skerrmag = np.sqrt(Skerr1*Skerr1+Skerr2*Skerr2+Skerr3*Skerr3)
    Skerrhat3 = Skerr3/Skerrmag
    Skerrhat2 = Skerr2/Skerrmag
    Skerrhat1 = Skerr1/Skerrmag
    a = Skerrmag
    n3 = z/r
    n2 = y/r
    n1 = x/r
    e33 = Skerrhat3
    e32 = Skerrhat2
    e31 = Skerrhat1
    xi3 = e31*n2-e32*n1
    xi2 = -e31*n3+e33*n1
    xi1 = e32*n3-e33*n2
    v3 = n1*xi2-n2*xi1
    v2 = n3*xi1-n1*xi3
    v1 = n2*xi3-n3*xi2
    costheta = e31*n1+e32*n2+e33*n3
    sin2theta = 1-costheta*costheta
    xisq = sin2theta
    w2 = a*a+r*r
    Sigma = r*r+a*a*costheta*costheta
    Dinv = 1+np.log(1+6*eta*u*u+2*(26-3*eta)*eta*u*u*u)
    omegatilde = 2*a*r
    K = 1.712-1.803949138004582*eta-39.77229225266885*eta*eta+103.16588921239249*eta*eta*eta
    etaKminus1 = eta*K-1
    Delta0 = K*(eta*K-2)
    Delta1 = -2*etaKminus1*(K+Delta0)
    Delta2 = np.true_divide(1,2)*Delta1*(Delta1-4*etaKminus1)-a*a*etaKminus1*etaKminus1*Delta0
    Delta3 = -np.true_divide(1,3)*Delta1*Delta1*Delta1+etaKminus1*Delta1*Delta1+Delta2*Delta1-2*etaKminus1*(Delta2-etaKminus1)-a*a*etaKminus1*etaKminus1*Delta1
    Delta4 = np.true_divide(1,12)*(6*a*a*(Delta1*Delta1-2*Delta2)*etaKminus1*etaKminus1+3*Delta1*Delta1*Delta1*Delta1-8*etaKminus1*Delta1*Delta1*Delta1-12*Delta2*Delta1*Delta1+12*(2*etaKminus1*Delta2+Delta3)*Delta1+12*(np.true_divide(94,3)-np.true_divide(41,32)*np.pi*np.pi)*etaKminus1*etaKminus1+6*(Delta2*Delta2-4*Delta3*etaKminus1))
    Delta5 = etaKminus1*etaKminus1*((np.true_divide(-4237,60)+np.true_divide(128,5)*EMgamma+np.true_divide(2275,512)*np.pi*np.pi-np.true_divide(1,3)*a*a*(Delta1*Delta1*Delta1-3*Delta1*Delta2+3*Delta3)-(Delta1*Delta1*Delta1*Delta1*Delta1-5*Delta1*Delta1*Delta1*Delta2+5*Delta1*Delta2*Delta2+5*Delta1*Delta1*Delta3-5*Delta2*Delta3-5*Delta1*Delta4)/(5*etaKminus1*etaKminus1)+(Delta1*Delta1*Delta1*Delta1-4*Delta1*Delta1*Delta2+2*Delta2*Delta2+4*Delta1*Delta3-4*Delta4)/(2*etaKminus1)+np.true_divide(256,5)*np.log(2)))
    Delta5l = etaKminus1*etaKminus1*np.true_divide(64,5)
    logarg = u*(Delta1+u*(Delta2+u*(Delta3+u*(Delta4+u*(Delta5+Delta5l*np.log(u))))))
    Deltaucalib = 1+eta*(Delta0+np.log(1+logarg))
    Deltaucalib = -eta*u*u*(Delta1+u*(2*Delta2+u*(3*Delta3+u*(4*Delta4+u*(5*(Delta5+Delta5l*np.log(u)))))))/(1+logarg)
    Deltaubar = a*a*u*u+(2*u+1/etaKminus1)/etaKminus1
    Deltaubar = -2*a*a*u*u*u-2*u*u/(etaKminus1)
    Deltau = Deltaubar*Deltaucalib
    Deltau = Deltaubar*Deltaucalib+Deltaubar*Deltaucalib
    Deltat = 2*r*Deltau+r*r*Deltau
    Deltat = r*r*Deltau
    Deltar = Deltat*Dinv
    Lambt = w2*w2-a*a*Deltat*sin2theta
    csi = np.sqrt(Deltar*Deltat)/w2
    csi1 = 1+(1-abs(1-tortoise))*(csi-1)
    csi2 = 1+(np.true_divide(1,2)-np.true_divide(1,2)*np.sign(np.true_divide(3,2)-tortoise))*(csi-1)
    prT = csi2*(p1*n1+p2*n2+p3*n3)
    phat3 = p3+prT*(1-1/csi1)*n3
    phat2 = p2+prT*(1-1/csi1)*n2
    phat1 = p1+prT*(1-1/csi1)*n1
    pdotxir = (phat1*xi1+phat2*xi2+phat3*xi3)*r
    pdotn = phat1*n1+phat2*n2+phat3*n3
    pdotvr = (phat1*v1+phat2*v2+phat3*v3)*r
    pphi = pdotxir
    Qcoeff2 = 1/(Sigma*sin2theta)
    Qcoeff1 = Sigma/(Lambt*sin2theta)
    DrSipn2 = Deltar*pdotn*pdotn/Sigma
    Q = 1+DrSipn2+Qcoeff1*pdotxir*pdotxir+Qcoeff2*pdotvr*pdotvr
    Qminus1 = Q-1
    Jtilde = np.sqrt(Deltar)
    exp2mu = Sigma
    expmu = np.sqrt(exp2mu)
    Brtilde = (np.sqrt(Deltar)*Deltat-2*Deltat)/(2*np.sqrt(Deltar*Deltat))
    Btilde = np.sqrt(Deltat)
    exp2nu = Deltat*Sigma/Lambt
    expnu = np.sqrt(exp2nu)
    omega = omegatilde/Lambt
    omegatilde = 2*a
    Lambt = 4*(a*a+r*r)*r-2*a*a*Deltat*sin2theta
    mucostheta = a*a*costheta/Sigma
    nucostheta = a*a*w2*costheta*(w2-Deltat)/(Lambt*Sigma)
    omegacostheta = -2*a*a*costheta*Deltat*omegatilde/(Lambt*Lambt)
    mur = r/Sigma-1/np.sqrt(Deltar)
    nur = r/Sigma+w2*(w2*Deltat-4*r*Deltat)/(2*Lambt*Deltat)
    omegar = (Lambt*omegatilde-Lambt*omegatilde)/(Lambt*Lambt)
    dSO = -74.71-156.*eta+627.5*eta*eta
    sigmacoeffTerm3 = eta*dSO*u*u*u
    sigmacoeffTerm2 = eta/(144*r*r)*(-896+r*(-436*Qminus1-96*DrSipn2+r*(-45*Qminus1*Qminus1+36*Qminus1*DrSipn2))+eta*(-336+r*(204*Qminus1-882*DrSipn2+r*(810*DrSipn2*DrSipn2-234*Qminus1*DrSipn2))))
    sigmacoeffTerm1 = eta/12*(-8/r+3*Qminus1-36*DrSipn2)
    sigmacoeff = sigmacoeffTerm1+sigmacoeffTerm2+sigmacoeffTerm3
    sigmastarcoeffTerm2 = eta/(72*r*r)*(706+r*(-206*Qminus1+282*DrSipn2+r*Qminus1*(96*DrSipn2-23*Qminus1))+eta*(-54+r*(120*Qminus1-324*DrSipn2+r*(360*DrSipn2*DrSipn2+Qminus1*(-126*DrSipn2-3*Qminus1)))))
    sigmastarcoeffTerm1 = eta/12*(14/r+4*Qminus1-30*DrSipn2)
    sigmastarcoeff = sigmastarcoeffTerm1+sigmastarcoeffTerm2
    Deltasigmastar3 = sigmastar3*sigmastarcoeff+sigma3*sigmacoeff
    Deltasigmastar2 = sigmastar2*sigmastarcoeff+sigma2*sigmacoeff
    Deltasigmastar1 = sigmastar1*sigmastarcoeff+sigma1*sigmacoeff
    Sstar3 = sigmastar3+Deltasigmastar3
    Sstar2 = sigmastar2+Deltasigmastar2
    Sstar1 = sigmastar1+Deltasigmastar1
    S3 = Sstar3
    S2 = Sstar2
    S1 = Sstar1
    Sstardotn = Sstar1*n1+Sstar2*n2+Sstar3*n3
    SdotSkerrhat = S1*Skerrhat1+S2*Skerrhat2+S3*Skerrhat3
    Sdotn = S1*n1+S2*n2+S3*n3
    Sdotv = S1*v1+S2*v2+S3*v3
    Sdotxi = S1*xi1+S2*xi2+S3*xi3
    HdsumTerm2 = 3*Sstardotn*Sstardotn
    HdsumTerm1 = Sstar1*Sstar1+Sstar2*Sstar2+Sstar3*Sstar3
    Hdsum = HdsumTerm1-HdsumTerm2
    Hdcoeff = np.true_divide(1,2)/(r*r*r)
    Q4 = 2*prT*prT*prT*prT*u*u*(4-3*eta)*eta
    gammappsum = Deltar/Sigma*pdotn*pdotn+1/Sigma*pdotvr*pdotvr/sin2theta+Sigma/Lambt/sin2theta*pdotxir*pdotxir
    Hnsradicand = 1+gammappsum+Q4
    alpha = np.sqrt(Deltat*Sigma/Lambt)
    betapsum = omegatilde*pphi/Lambt
    HssTerm3 = expmu*expnu*pdotxir*(Jtilde*pdotn*Sdotxi*Btilde-expmu*expnu*pdotxir*Sdotn)+(pdotvr*(Sdotn*pdotvr-Jtilde*pdotn*Sdotv)-exp2mu*(np.sqrt(Q)+Q)*Sdotn*xisq)*Btilde*Btilde
    HssTerm3coeff = omegacostheta/(2*exp2mu*expmu*expnu*Btilde*(Q+np.sqrt(Q)))
    HssTerm2 = expmu*pdotxir*(expmu*exp2nu*pdotxir*Sdotv-expnu*pdotvr*Sdotxi*Btilde)+xisq*Btilde*Btilde*(exp2mu*(np.sqrt(Q)+Q)*Sdotv+Jtilde*pdotn*(pdotvr*Sdotn-Jtilde*pdotn*Sdotv))
    HssTerm2coeff = Jtilde*omegar/(2*exp2mu*expmu*expnu*Btilde*(Q+np.sqrt(Q))*xisq)
    HssTerm1 = omega*SdotSkerrhat
    Hss = HssTerm1+HssTerm2coeff*HssTerm2+HssTerm3coeff*HssTerm3
    HsoTerm2c = Jtilde*Brtilde*expmu*expnu*pdotxir*(np.sqrt(Q)+1)*Sdotv
    HsoTerm2b = expmu*expnu*pdotxir*(2*np.sqrt(Q)+1)*(Jtilde*nur*Sdotv-nucostheta*Sdotn*xisq)*Btilde
    HsoTerm2a = Sdotxi*Jtilde*(mur*pdotvr*(np.sqrt(Q)+1)-mucostheta*pdotn*xisq-np.sqrt(Q)*(nur*pdotvr+(mucostheta-nucostheta)*pdotn*xisq))*Btilde*Btilde
    HsoTerm2 = HsoTerm2a+HsoTerm2b-HsoTerm2c
    HsoTerm2coeff = expnu/(exp2mu*Btilde*Btilde*(Q+np.sqrt(Q))*xisq)
    HsoTerm1 = exp2nu*(expmu*expnu-Btilde)*pdotxir*SdotSkerrhat/(expmu*Btilde*Btilde*np.sqrt(Q)*xisq)
    Hso = HsoTerm1+HsoTerm2coeff*HsoTerm2
    Hd = Hdcoeff*Hdsum
    Hns = betapsum+alpha*np.sqrt(Hnsradicand)
    Hs = Hso+Hss
    dSS = 8.127-154.2*eta+830.8*eta*eta
    Heff = Hs+Hns-Hd+dSS*eta*u*u*u*u*(S1x*S1x+S1y*S1y+S1z*S1z+S2x*S2x+S2y*S2y+S2z*S2z)
    r_x = x/np.sqrt(x**2+y**2+z**2)
    u_x = -r_x/r**2
    n3_x = -r_x*z/r**2
    n2_x = -r_x*y/r**2
    n1_x = 1/r-r_x*x/r**2
    xi3_x = e31*n2_x-e32*n1_x
    xi2_x = -e31*n3_x+e33*n1_x
    xi1_x = e32*n3_x-e33*n2_x
    v3_x = n1*xi2_x+n1_x*xi2-n2*xi1_x-n2_x*xi1
    v2_x = -n1*xi3_x-n1_x*xi3+n3*xi1_x+n3_x*xi1
    v1_x = n2*xi3_x+n2_x*xi3-n3*xi2_x-n3_x*xi2
    costheta_x = e31*n1_x+e32*n2_x+e33*n3_x
    sin2theta_x = -2*costheta*costheta_x
    xisq_x = sin2theta_x
    w2_x = 2*r*r_x
    Sigma_x = 2*a**2*costheta*costheta_x+2*r*r_x
    Dinv_x = (3*eta*u**2*u_x*(-6*eta+52)+12*eta*u*u_x)/(eta*u**3*(-6*eta+52)+6*eta*u**2+1)
    omegatilde_x = 2*a*r_x
    logarg_x = u*(u*(u*(u*(Delta5l*u_x+u_x*(Delta5+Delta5l*np.log(u)))+u_x*(Delta4+u*(Delta5+Delta5l*np.log(u))))+u_x*(Delta3+u*(Delta4+u*(Delta5+Delta5l*np.log(u)))))+u_x*(Delta2+u*(Delta3+u*(Delta4+u*(Delta5+Delta5l*np.log(u))))))+u_x*(Delta1+u*(Delta2+u*(Delta3+u*(Delta4+u*(Delta5+Delta5l*np.log(u))))))
    Deltaucalib_x = eta*logarg_x/(logarg+1)
    Deltaucalib_x_x = eta*logarg_x*u**2*(Delta1+u*(2*Delta2+u*(3*Delta3+u*(4*Delta4+u*(5*Delta5+5*Delta5l*np.log(u))))))/(logarg+1)**2-eta*u**2*(u*(u*(u*(5*Delta5l*u_x+u_x*(5*Delta5+5*Delta5l*np.log(u)))+u_x*(4*Delta4+u*(5*Delta5+5*Delta5l*np.log(u))))+u_x*(3*Delta3+u*(4*Delta4+u*(5*Delta5+5*Delta5l*np.log(u)))))+u_x*(2*Delta2+u*(3*Delta3+u*(4*Delta4+u*(5*Delta5+5*Delta5l*np.log(u))))))/(logarg+1)-2*eta*u*u_x*(Delta1+u*(2*Delta2+u*(3*Delta3+u*(4*Delta4+u*(5*Delta5+5*Delta5l*np.log(u))))))/(logarg+1)
    Deltaubar_x = 2*a**2*u*u_x+2*u_x/etaKminus1
    Deltaubar_x_x = -6*a**2*u**2*u_x-4*u*u_x/etaKminus1
    Deltau_x = Deltaubar*Deltaucalib_x+Deltaubar_x*Deltaucalib
    Deltau_x_x = Deltaubar*Deltaucalib_x_x+2*Deltaubar_x*Deltaucalib_x+Deltaubar_x_x*Deltaucalib
    Deltat_x_x = 2*Deltau*r_x+2*Deltau_x*r*r_x+2*Deltau_x*r+Deltau_x_x*r**2
    Deltat_x = 2*Deltau*r*r_x+Deltau_x*r**2
    return "2*Deltau*r*r_x+Deltau_x*r**2"
    Deltar_x = Deltat*Dinv_x+Deltat_x*Dinv
    Lambt_x = -Deltat*a**2*sin2theta_x-Deltat_x*a**2*sin2theta+2*w2*w2_x
    csi_x = -w2_x*np.sqrt(Deltar*Deltat)/w2**2+np.sqrt(Deltar*Deltat)*(Deltar*Deltat_x/2+Deltar_x*Deltat/2)/(Deltar*Deltat*w2)
    csi1_x = csi_x*(-abs(-tortoise+1)+1)
    csi2_x = csi_x*(-np.sign(-tortoise+3/2)/2+1/2)
    prT_x = csi2*(n1_x*p1+n2_x*p2+n3_x*p3)+csi2_x*(n1*p1+n2*p2+n3*p3)
    phat3_x = n3*prT_x*(1-1/csi1)+n3_x*prT*(1-1/csi1)+csi1_x*n3*prT/csi1**2
    phat2_x = n2*prT_x*(1-1/csi1)+n2_x*prT*(1-1/csi1)+csi1_x*n2*prT/csi1**2
    phat1_x = n1*prT_x*(1-1/csi1)+n1_x*prT*(1-1/csi1)+csi1_x*n1*prT/csi1**2
    pdotxir_x = r*(phat1*xi1_x+phat1_x*xi1+phat2*xi2_x+phat2_x*xi2+phat3*xi3_x+phat3_x*xi3)+r_x*(phat1*xi1+phat2*xi2+phat3*xi3)
    pdotn_x = n1*phat1_x+n1_x*phat1+n2*phat2_x+n2_x*phat2+n3*phat3_x+n3_x*phat3
    pdotvr_x = r*(phat1*v1_x+phat1_x*v1+phat2*v2_x+phat2_x*v2+phat3*v3_x+phat3_x*v3)+r_x*(phat1*v1+phat2*v2+phat3*v3)
    pphi_x = pdotxir_x
    Qcoeff2_x = -sin2theta_x/(Sigma*sin2theta**2)-Sigma_x/(Sigma**2*sin2theta)
    Qcoeff1_x = -Sigma*sin2theta_x/(Lambt*sin2theta**2)+Sigma_x/(Lambt*sin2theta)-Lambt_x*Sigma/(Lambt**2*sin2theta)
    DrSipn2_x = 2*Deltar*pdotn*pdotn_x/Sigma-Deltar*Sigma_x*pdotn**2/Sigma**2+Deltar_x*pdotn**2/Sigma
    Q_x = DrSipn2_x+2*Qcoeff1*pdotxir*pdotxir_x+Qcoeff1_x*pdotxir**2+2*Qcoeff2*pdotvr*pdotvr_x+Qcoeff2_x*pdotvr**2
    Qminus1_x = Q_x
    Jtilde_x = Deltar_x/(2*np.sqrt(Deltar))
    exp2mu_x = Sigma_x
    expmu_x = exp2mu_x/(2*np.sqrt(exp2mu))
    Brtilde_x = (np.sqrt(Deltar)*Deltat_x_x-2*Deltat_x+Deltar_x*Deltat_x/(2*np.sqrt(Deltar)))/(2*np.sqrt(Deltar*Deltat))+(np.sqrt(Deltar)*Deltat_x-2*Deltat)*(-Deltar*Deltat_x/2-Deltar_x*Deltat/2)/(2*Deltar*Deltat*np.sqrt(Deltar*Deltat))
    Btilde_x = Deltat_x/(2*np.sqrt(Deltat))
    exp2nu_x = Deltat*Sigma_x/Lambt-Deltat*Lambt_x*Sigma/Lambt**2+Deltat_x*Sigma/Lambt
    expnu_x = exp2nu_x/(2*np.sqrt(exp2nu))
    omega_x = omegatilde_x/Lambt-Lambt_x*omegatilde/Lambt**2
    Lambt_x_x = -2*Deltat_x*a**2*sin2theta_x-2*Deltat_x_x*a**2*sin2theta+8*r**2*r_x+r_x*(4*a**2+4*r**2)
    mucostheta_x = a**2*costheta_x/Sigma-Sigma_x*a**2*costheta/Sigma**2
    nucostheta_x = a**2*costheta*w2*(-Deltat_x+w2_x)/(Lambt*Sigma)+a**2*costheta*w2_x*(-Deltat+w2)/(Lambt*Sigma)+a**2*costheta_x*w2*(-Deltat+w2)/(Lambt*Sigma)-Sigma_x*a**2*costheta*w2*(-Deltat+w2)/(Lambt*Sigma**2)-Lambt_x*a**2*costheta*w2*(-Deltat+w2)/(Lambt**2*Sigma)
    omegacostheta_x = -2*Deltat*a**2*costheta*omegatilde_x/Lambt**2-2*Deltat*a**2*costheta_x*omegatilde/Lambt**2+4*Deltat*Lambt_x*a**2*costheta*omegatilde/Lambt**3-2*Deltat_x*a**2*costheta*omegatilde/Lambt**2
    mur_x = r_x/Sigma-Sigma_x*r/Sigma**2+Deltar_x/(2*Deltar**(3/2))
    nur_x = r_x/Sigma-Sigma_x*r/Sigma**2+w2*(-4*Deltat*r_x-4*Deltat_x*r+Deltat_x*w2_x+Deltat_x_x*w2)/(2*Deltat*Lambt)+w2_x*(-4*Deltat*r+Deltat_x*w2)/(2*Deltat*Lambt)-Lambt_x*w2*(-4*Deltat*r+Deltat_x*w2)/(2*Deltat*Lambt**2)-Deltat_x*w2*(-4*Deltat*r+Deltat_x*w2)/(2*Deltat**2*Lambt)
    omegar_x = -Lambt_x_x*omegatilde/Lambt**2+Lambt_x*(-2*Lambt*omegatilde_x+2*Lambt_x*omegatilde)/Lambt**3
    sigmacoeffTerm3_x = 3*dSO*eta*u**2*u_x
    sigmacoeffTerm2_x = eta*(eta*(r*(-882*DrSipn2_x+204*Qminus1_x+r*(1620*DrSipn2*DrSipn2_x-234*DrSipn2*Qminus1_x-234*DrSipn2_x*Qminus1)+r_x*(810*DrSipn2**2-234*DrSipn2*Qminus1))+r_x*(-882*DrSipn2+204*Qminus1+r*(810*DrSipn2**2-234*DrSipn2*Qminus1)))+r*(-96*DrSipn2_x-436*Qminus1_x+r*(36*DrSipn2*Qminus1_x+36*DrSipn2_x*Qminus1-90*Qminus1*Qminus1_x)+r_x*(36*DrSipn2*Qminus1-45*Qminus1**2))+r_x*(-96*DrSipn2-436*Qminus1+r*(36*DrSipn2*Qminus1-45*Qminus1**2)))/(144*r**2)-eta*r_x*(eta*(r*(-882*DrSipn2+204*Qminus1+r*(810*DrSipn2**2-234*DrSipn2*Qminus1))-336)+r*(-96*DrSipn2-436*Qminus1+r*(36*DrSipn2*Qminus1-45*Qminus1**2))-896)/(72*r**3)
    sigmacoeffTerm1_x = eta*(-36*DrSipn2_x+3*Qminus1_x+8*r_x/r**2)/12
    sigmacoeff_x = sigmacoeffTerm1_x+sigmacoeffTerm2_x+sigmacoeffTerm3_x
    sigmastarcoeffTerm2_x = eta*(eta*(r*(-324*DrSipn2_x+120*Qminus1_x+r*(720*DrSipn2*DrSipn2_x+Qminus1*(-126*DrSipn2_x-3*Qminus1_x)+Qminus1_x*(-126*DrSipn2-3*Qminus1))+r_x*(360*DrSipn2**2+Qminus1*(-126*DrSipn2-3*Qminus1)))+r_x*(-324*DrSipn2+120*Qminus1+r*(360*DrSipn2**2+Qminus1*(-126*DrSipn2-3*Qminus1))))+r*(282*DrSipn2_x+Qminus1*r*(96*DrSipn2_x-23*Qminus1_x)+Qminus1*r_x*(96*DrSipn2-23*Qminus1)+Qminus1_x*r*(96*DrSipn2-23*Qminus1)-206*Qminus1_x)+r_x*(282*DrSipn2+Qminus1*r*(96*DrSipn2-23*Qminus1)-206*Qminus1))/(72*r**2)-eta*r_x*(eta*(r*(-324*DrSipn2+120*Qminus1+r*(360*DrSipn2**2+Qminus1*(-126*DrSipn2-3*Qminus1)))-54)+r*(282*DrSipn2+Qminus1*r*(96*DrSipn2-23*Qminus1)-206*Qminus1)+706)/(36*r**3)
    sigmastarcoeffTerm1_x = eta*(-30*DrSipn2_x+4*Qminus1_x-14*r_x/r**2)/12
    sigmastarcoeff_x = sigmastarcoeffTerm1_x+sigmastarcoeffTerm2_x
    Deltasigmastar3_x = sigma3*sigmacoeff_x+sigmastar3*sigmastarcoeff_x
    Deltasigmastar2_x = sigma2*sigmacoeff_x+sigmastar2*sigmastarcoeff_x
    Deltasigmastar1_x = sigma1*sigmacoeff_x+sigmastar1*sigmastarcoeff_x
    Sstar3_x = Deltasigmastar3_x
    Sstar2_x = Deltasigmastar2_x
    Sstar1_x = Deltasigmastar1_x
    S3_x = Sstar3_x
    S2_x = Sstar2_x
    S1_x = Sstar1_x
    Sstardotn_x = Sstar1*n1_x+Sstar1_x*n1+Sstar2*n2_x+Sstar2_x*n2+Sstar3*n3_x+Sstar3_x*n3
    SdotSkerrhat_x = S1_x*Skerrhat1+S2_x*Skerrhat2+S3_x*Skerrhat3
    Sdotn_x = S1*n1_x+S1_x*n1+S2*n2_x+S2_x*n2+S3*n3_x+S3_x*n3
    Sdotv_x = S1*v1_x+S1_x*v1+S2*v2_x+S2_x*v2+S3*v3_x+S3_x*v3
    Sdotxi_x = S1*xi1_x+S1_x*xi1+S2*xi2_x+S2_x*xi2+S3*xi3_x+S3_x*xi3
    HdsumTerm2_x = 6*Sstardotn*Sstardotn_x
    HdsumTerm1_x = 2*Sstar1*Sstar1_x+2*Sstar2*Sstar2_x+2*Sstar3*Sstar3_x
    Hdsum_x = HdsumTerm1_x-HdsumTerm2_x
    Hdcoeff_x = -3*r_x/(2*r**4)
    Q4_x = 4*eta*prT**4*u*u_x*(-3*eta+4)+8*eta*prT**3*prT_x*u**2*(-3*eta+4)
    gammappsum_x = 2*Deltar*pdotn*pdotn_x/Sigma-Deltar*Sigma_x*pdotn**2/Sigma**2+Deltar_x*pdotn**2/Sigma-pdotvr**2*sin2theta_x/(Sigma*sin2theta**2)+2*pdotvr*pdotvr_x/(Sigma*sin2theta)-Sigma_x*pdotvr**2/(Sigma**2*sin2theta)-Sigma*pdotxir**2*sin2theta_x/(Lambt*sin2theta**2)+2*Sigma*pdotxir*pdotxir_x/(Lambt*sin2theta)+Sigma_x*pdotxir**2/(Lambt*sin2theta)-Lambt_x*Sigma*pdotxir**2/(Lambt**2*sin2theta)
    Hnsradicand_x = Q4_x+gammappsum_x
    alpha_x = Lambt*np.sqrt(Deltat*Sigma/Lambt)*(Deltat*Sigma_x/(2*Lambt)-Deltat*Lambt_x*Sigma/(2*Lambt**2)+Deltat_x*Sigma/(2*Lambt))/(Deltat*Sigma)
    betapsum_x = omegatilde*pphi_x/Lambt+omegatilde_x*pphi/Lambt-Lambt_x*omegatilde*pphi/Lambt**2
    HssTerm3_x = Btilde**2*(Sdotn*exp2mu*xisq*(-Q_x-Q_x/(2*np.sqrt(Q)))+Sdotn*exp2mu*xisq_x*(-np.sqrt(Q)-Q)+Sdotn*exp2mu_x*xisq*(-np.sqrt(Q)-Q)+Sdotn_x*exp2mu*xisq*(-np.sqrt(Q)-Q)+pdotvr*(-Jtilde*Sdotv*pdotn_x-Jtilde*Sdotv_x*pdotn-Jtilde_x*Sdotv*pdotn+Sdotn*pdotvr_x+Sdotn_x*pdotvr)+pdotvr_x*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr))+Btilde*Btilde_x*(-2*Sdotn*exp2mu*xisq*(np.sqrt(Q)+Q)+2*pdotvr*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr))+expmu*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn_x+Btilde*Jtilde*Sdotxi_x*pdotn+Btilde*Jtilde_x*Sdotxi*pdotn+Btilde_x*Jtilde*Sdotxi*pdotn-Sdotn*expmu*expnu*pdotxir_x-Sdotn*expmu*expnu_x*pdotxir-Sdotn*expmu_x*expnu*pdotxir-Sdotn_x*expmu*expnu*pdotxir)+expmu*expnu*pdotxir_x*(Btilde*Jtilde*Sdotxi*pdotn-Sdotn*expmu*expnu*pdotxir)+expmu*expnu_x*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn-Sdotn*expmu*expnu*pdotxir)+expmu_x*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn-Sdotn*expmu*expnu*pdotxir)
    HssTerm3coeff_x = omegacostheta*(-Q_x-Q_x/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q)+Q)**2)+omegacostheta_x/(Btilde*exp2mu*expmu*expnu*(2*np.sqrt(Q)+2*Q))-expnu_x*omegacostheta/(Btilde*exp2mu*expmu*expnu**2*(2*np.sqrt(Q)+2*Q))-expmu_x*omegacostheta/(Btilde*exp2mu*expmu**2*expnu*(2*np.sqrt(Q)+2*Q))-exp2mu_x*omegacostheta/(Btilde*exp2mu**2*expmu*expnu*(2*np.sqrt(Q)+2*Q))-Btilde_x*omegacostheta/(Btilde**2*exp2mu*expmu*expnu*(2*np.sqrt(Q)+2*Q))
    HssTerm2_x = Btilde**2*xisq*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotn_x-Jtilde*Sdotv_x*pdotn-Jtilde_x*Sdotv*pdotn+Sdotn*pdotvr_x+Sdotn_x*pdotvr)+Jtilde*pdotn_x*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr)+Jtilde_x*pdotn*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr)+Sdotv*exp2mu*(Q_x+Q_x/(2*np.sqrt(Q)))+Sdotv*exp2mu_x*(np.sqrt(Q)+Q)+Sdotv_x*exp2mu*(np.sqrt(Q)+Q))+Btilde**2*xisq_x*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr)+Sdotv*exp2mu*(np.sqrt(Q)+Q))+Btilde*Btilde_x*xisq*(2*Jtilde*pdotn*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr)+2*Sdotv*exp2mu*(np.sqrt(Q)+Q))+expmu*pdotxir*(-Btilde*Sdotxi*expnu*pdotvr_x-Btilde*Sdotxi*expnu_x*pdotvr-Btilde*Sdotxi_x*expnu*pdotvr-Btilde_x*Sdotxi*expnu*pdotvr+Sdotv*exp2nu*expmu*pdotxir_x+Sdotv*exp2nu*expmu_x*pdotxir+Sdotv*exp2nu_x*expmu*pdotxir+Sdotv_x*exp2nu*expmu*pdotxir)+expmu*pdotxir_x*(-Btilde*Sdotxi*expnu*pdotvr+Sdotv*exp2nu*expmu*pdotxir)+expmu_x*pdotxir*(-Btilde*Sdotxi*expnu*pdotvr+Sdotv*exp2nu*expmu*pdotxir)
    HssTerm2coeff_x = Jtilde*omegar*(-Q_x-Q_x/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q)+Q)**2)-Jtilde*omegar*xisq_x/(Btilde*exp2mu*expmu*expnu*xisq**2*(2*np.sqrt(Q)+2*Q))+Jtilde*omegar_x/(Btilde*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q)+2*Q))-Jtilde*expnu_x*omegar/(Btilde*exp2mu*expmu*expnu**2*xisq*(2*np.sqrt(Q)+2*Q))-Jtilde*expmu_x*omegar/(Btilde*exp2mu*expmu**2*expnu*xisq*(2*np.sqrt(Q)+2*Q))-Jtilde*exp2mu_x*omegar/(Btilde*exp2mu**2*expmu*expnu*xisq*(2*np.sqrt(Q)+2*Q))+Jtilde_x*omegar/(Btilde*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q)+2*Q))-Btilde_x*Jtilde*omegar/(Btilde**2*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q)+2*Q))
    HssTerm1_x = SdotSkerrhat*omega_x+SdotSkerrhat_x*omega
    Hss_x = HssTerm1_x+HssTerm2*HssTerm2coeff_x+HssTerm2coeff*HssTerm2_x+HssTerm3*HssTerm3coeff_x+HssTerm3coeff*HssTerm3_x
    HsoTerm2c_x = Brtilde*Jtilde*Sdotv*expmu*expnu*pdotxir_x*(np.sqrt(Q)+1)+Brtilde*Jtilde*Sdotv*expmu*expnu_x*pdotxir*(np.sqrt(Q)+1)+Brtilde*Jtilde*Sdotv*expmu_x*expnu*pdotxir*(np.sqrt(Q)+1)+Brtilde*Jtilde*Sdotv_x*expmu*expnu*pdotxir*(np.sqrt(Q)+1)+Brtilde*Jtilde*Q_x*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q))+Brtilde*Jtilde_x*Sdotv*expmu*expnu*pdotxir*(np.sqrt(Q)+1)+Brtilde_x*Jtilde*Sdotv*expmu*expnu*pdotxir*(np.sqrt(Q)+1)
    HsoTerm2b_x = Btilde*expmu*expnu*pdotxir*(2*np.sqrt(Q)+1)*(Jtilde*Sdotv*nur_x+Jtilde*Sdotv_x*nur+Jtilde_x*Sdotv*nur-Sdotn*nucostheta*xisq_x-Sdotn*nucostheta_x*xisq-Sdotn_x*nucostheta*xisq)+Btilde*expmu*expnu*pdotxir_x*(2*np.sqrt(Q)+1)*(Jtilde*Sdotv*nur-Sdotn*nucostheta*xisq)+Btilde*expmu*expnu_x*pdotxir*(2*np.sqrt(Q)+1)*(Jtilde*Sdotv*nur-Sdotn*nucostheta*xisq)+Btilde*expmu_x*expnu*pdotxir*(2*np.sqrt(Q)+1)*(Jtilde*Sdotv*nur-Sdotn*nucostheta*xisq)+Btilde*Q_x*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur-Sdotn*nucostheta*xisq)/np.sqrt(Q)+Btilde_x*expmu*expnu*pdotxir*(2*np.sqrt(Q)+1)*(Jtilde*Sdotv*nur-Sdotn*nucostheta*xisq)
    HsoTerm2a_x = Btilde**2*Jtilde*Sdotxi*(-np.sqrt(Q)*(nur*pdotvr_x+nur_x*pdotvr+pdotn*xisq*(mucostheta_x-nucostheta_x)+pdotn*xisq_x*(mucostheta-nucostheta)+pdotn_x*xisq*(mucostheta-nucostheta))-mucostheta*pdotn*xisq_x-mucostheta*pdotn_x*xisq-mucostheta_x*pdotn*xisq+mur*pdotvr_x*(np.sqrt(Q)+1)+mur_x*pdotvr*(np.sqrt(Q)+1)+Q_x*mur*pdotvr/(2*np.sqrt(Q))+Q_x*(-nur*pdotvr-pdotn*xisq*(mucostheta-nucostheta))/(2*np.sqrt(Q)))+Btilde**2*Jtilde*Sdotxi_x*(np.sqrt(Q)*(-nur*pdotvr-pdotn*xisq*(mucostheta-nucostheta))-mucostheta*pdotn*xisq+mur*pdotvr*(np.sqrt(Q)+1))+Btilde**2*Jtilde_x*Sdotxi*(np.sqrt(Q)*(-nur*pdotvr-pdotn*xisq*(mucostheta-nucostheta))-mucostheta*pdotn*xisq+mur*pdotvr*(np.sqrt(Q)+1))+Btilde*Btilde_x*Jtilde*Sdotxi*(2*np.sqrt(Q)*(-nur*pdotvr-pdotn*xisq*(mucostheta-nucostheta))-2*mucostheta*pdotn*xisq+2*mur*pdotvr*(np.sqrt(Q)+1))
    HsoTerm2_x = HsoTerm2a_x+HsoTerm2b_x-HsoTerm2c_x
    return HsoTerm2a_x
    HsoTerm2coeff_x = expnu*(-Q_x-Q_x/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q)+Q)**2)-expnu*xisq_x/(Btilde**2*exp2mu*xisq**2*(np.sqrt(Q)+Q))+expnu_x/(Btilde**2*exp2mu*xisq*(np.sqrt(Q)+Q))-exp2mu_x*expnu/(Btilde**2*exp2mu**2*xisq*(np.sqrt(Q)+Q))-2*Btilde_x*expnu/(Btilde**3*exp2mu*xisq*(np.sqrt(Q)+Q))
    HsoTerm1_x = SdotSkerrhat*exp2nu*pdotxir*(-Btilde_x+expmu*expnu_x+expmu_x*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq)+SdotSkerrhat*exp2nu*pdotxir*xisq_x*(Btilde-expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq**2)+SdotSkerrhat*exp2nu*pdotxir_x*(-Btilde+expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq)-SdotSkerrhat*exp2nu*expmu_x*pdotxir*(-Btilde+expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu**2*xisq)+SdotSkerrhat*exp2nu_x*pdotxir*(-Btilde+expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq)+SdotSkerrhat_x*exp2nu*pdotxir*(-Btilde+expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq)-Q_x*SdotSkerrhat*exp2nu*pdotxir*(-Btilde+expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq)-Btilde_x*SdotSkerrhat*exp2nu*pdotxir*(-2*Btilde+2*expmu*expnu)/(Btilde**3*np.sqrt(Q)*expmu*xisq)
    Hso_x = HsoTerm1_x+HsoTerm2*HsoTerm2coeff_x+HsoTerm2coeff*HsoTerm2_x
    Hd_x = Hdcoeff*Hdsum_x+Hdcoeff_x*Hdsum
    Hns_x = np.sqrt(Hnsradicand)*alpha_x+betapsum_x+Hnsradicand_x*alpha/(2*np.sqrt(Hnsradicand))
    Hs_x = Hso_x+Hss_x
    Heff_x = -Hd_x+Hns_x+Hs_x+4*dSS*eta*u**3*u_x*(S1x**2+S1y**2+S1z**2+S2x**2+S2y**2+S2z**2)
    prTprm_p2 = csi2*n2
    phat3prm_p2 = n3*prTprm_p2*(1 - 1/csi1)
    phat2prm_p2 = n2*prTprm_p2*(1 - 1/csi1) + 1
    phat1prm_p2 = n1*prTprm_p2*(1 - 1/csi1)
    pdotxirprm_p2 = r*(phat1prm_p2*xi1 + phat2prm_p2*xi2 + phat3prm_p2*xi3)
    pdotnprm_p2 = n1*phat1prm_p2 + n2*phat2prm_p2 + n3*phat3prm_p2
    pdotvrprm_p2 = r*(phat1prm_p2*v1 + phat2prm_p2*v2 + phat3prm_p2*v3)
    pphiprm_p2 = pdotxirprm_p2
    DrSipn2prm_p2 = 2*Deltar*pdotn*pdotnprm_p2/Sigma
    Qprm_p2 = DrSipn2prm_p2 + 2*Qcoeff1*pdotxir*pdotxirprm_p2 + 2*Qcoeff2*pdotvr*pdotvrprm_p2
    Qminus1prm_p2 = Qprm_p2
    sigmacoeffTerm2prm_p2 = eta*(eta*r*(-882*DrSipn2prm_p2 + 204*Qminus1prm_p2 + r*(1620*DrSipn2*DrSipn2prm_p2 - 234*DrSipn2*Qminus1prm_p2 - 234*DrSipn2prm_p2*Qminus1)) + r*(-96*DrSipn2prm_p2 - 436*Qminus1prm_p2 + r*(36*DrSipn2*Qminus1prm_p2 + 36*DrSipn2prm_p2*Qminus1 - 90*Qminus1*Qminus1prm_p2)))/(144*r**2)
    sigmacoeffTerm1prm_p2 = eta*(-36*DrSipn2prm_p2 + 3*Qminus1prm_p2)/12
    sigmacoeffprm_p2 = sigmacoeffTerm1prm_p2 + sigmacoeffTerm2prm_p2
    sigmastarcoeffTerm2prm_p2 = eta*(eta*r*(-324*DrSipn2prm_p2 + 120*Qminus1prm_p2 + r*(720*DrSipn2*DrSipn2prm_p2 + Qminus1*(-126*DrSipn2prm_p2 - 3*Qminus1prm_p2) + Qminus1prm_p2*(-126*DrSipn2 - 3*Qminus1))) + r*(282*DrSipn2prm_p2 + Qminus1*r*(96*DrSipn2prm_p2 - 23*Qminus1prm_p2) + Qminus1prm_p2*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1prm_p2))/(72*r**2)
    sigmastarcoeffTerm1prm_p2 = eta*(-30*DrSipn2prm_p2 + 4*Qminus1prm_p2)/12
    sigmastarcoeffprm_p2 = sigmastarcoeffTerm1prm_p2 + sigmastarcoeffTerm2prm_p2
    Deltasigmastar3prm_p2 = sigma3*sigmacoeffprm_p2 + sigmastar3*sigmastarcoeffprm_p2
    Deltasigmastar2prm_p2 = sigma2*sigmacoeffprm_p2 + sigmastar2*sigmastarcoeffprm_p2
    Deltasigmastar1prm_p2 = sigma1*sigmacoeffprm_p2 + sigmastar1*sigmastarcoeffprm_p2
    Sstar3prm_p2 = Deltasigmastar3prm_p2
    Sstar2prm_p2 = Deltasigmastar2prm_p2
    Sstar1prm_p2 = Deltasigmastar1prm_p2
    S3prm_p2 = Sstar3prm_p2
    S2prm_p2 = Sstar2prm_p2
    S1prm_p2 = Sstar1prm_p2
    Sstardotnprm_p2 = Sstar1prm_p2*n1 + Sstar2prm_p2*n2 + Sstar3prm_p2*n3
    SdotSkerrhatprm_p2 = S1prm_p2*Skerrhat1 + S2prm_p2*Skerrhat2 + S3prm_p2*Skerrhat3
    Sdotnprm_p2 = S1prm_p2*n1 + S2prm_p2*n2 + S3prm_p2*n3
    Sdotvprm_p2 = S1prm_p2*v1 + S2prm_p2*v2 + S3prm_p2*v3
    Sdotxiprm_p2 = S1prm_p2*xi1 + S2prm_p2*xi2 + S3prm_p2*xi3
    HdsumTerm2prm_p2 = 6*Sstardotn*Sstardotnprm_p2
    HdsumTerm1prm_p2 = 2*Sstar1*Sstar1prm_p2 + 2*Sstar2*Sstar2prm_p2 + 2*Sstar3*Sstar3prm_p2
    Hdsumprm_p2 = HdsumTerm1prm_p2 - HdsumTerm2prm_p2
    Q4prm_p2 = 8*eta*prT**3*prTprm_p2*u**2*(-3*eta + 4)
    gammappsumprm_p2 = 2*Deltar*pdotn*pdotnprm_p2/Sigma + 2*pdotvr*pdotvrprm_p2/(Sigma*sin2theta) + 2*Sigma*pdotxir*pdotxirprm_p2/(Lambt*sin2theta)
    Hnsradicandprm_p2 = Q4prm_p2 + gammappsumprm_p2
    betapsumprm_p2 = omegatilde*pphiprm_p2/Lambt
    HssTerm3prm_p2 = Btilde**2*(Sdotn*exp2mu*xisq*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q))) + Sdotnprm_p2*exp2mu*xisq*(-np.sqrt(Q) - Q) + pdotvr*(-Jtilde*Sdotv*pdotnprm_p2 - Jtilde*Sdotvprm_p2*pdotn + Sdotn*pdotvrprm_p2 + Sdotnprm_p2*pdotvr) + pdotvrprm_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr)) + expmu*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotnprm_p2 + Btilde*Jtilde*Sdotxiprm_p2*pdotn - Sdotn*expmu*expnu*pdotxirprm_p2 - Sdotnprm_p2*expmu*expnu*pdotxir) + expmu*expnu*pdotxirprm_p2*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir)
    HssTerm3coeffprm_p2 = omegacostheta*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**2)
    HssTerm2prm_p2 = Btilde**2*xisq*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotnprm_p2 - Jtilde*Sdotvprm_p2*pdotn + Sdotn*pdotvrprm_p2 + Sdotnprm_p2*pdotvr) + Jtilde*pdotnprm_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Sdotv*exp2mu*(Qprm_p2 + Qprm_p2/(2*np.sqrt(Q))) + Sdotvprm_p2*exp2mu*(np.sqrt(Q) + Q)) + expmu*pdotxir*(-Btilde*Sdotxi*expnu*pdotvrprm_p2 - Btilde*Sdotxiprm_p2*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxirprm_p2 + Sdotvprm_p2*exp2nu*expmu*pdotxir) + expmu*pdotxirprm_p2*(-Btilde*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir)
    HssTerm2coeffprm_p2 = Jtilde*omegar*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**2)
    HssTerm1prm_p2 = SdotSkerrhatprm_p2*omega
    Hssprm_p2 = HssTerm1prm_p2 + HssTerm2*HssTerm2coeffprm_p2 + HssTerm2coeff*HssTerm2prm_p2 + HssTerm3*HssTerm3coeffprm_p2 + HssTerm3coeff*HssTerm3prm_p2
    HsoTerm2cprm_p2 = Brtilde*Jtilde*Sdotv*expmu*expnu*pdotxirprm_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotvprm_p2*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Qprm_p2*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q))
    HsoTerm2bprm_p2 = Btilde*expmu*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotvprm_p2*nur - Sdotnprm_p2*nucostheta*xisq) + Btilde*expmu*expnu*pdotxirprm_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*Qprm_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q)
    HsoTerm2aprm_p2 = Btilde**2*Jtilde*Sdotxi*(-np.sqrt(Q)*(nur*pdotvrprm_p2 + pdotnprm_p2*xisq*(mucostheta - nucostheta)) - mucostheta*pdotnprm_p2*xisq + mur*pdotvrprm_p2*(np.sqrt(Q) + 1) + Qprm_p2*mur*pdotvr/(2*np.sqrt(Q)) + Qprm_p2*(-nur*pdotvr - pdotn*xisq*(mucostheta - nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtilde*Sdotxiprm_p2*(np.sqrt(Q)*(-nur*pdotvr - pdotn*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn*xisq + mur*pdotvr*(np.sqrt(Q) + 1))
    HsoTerm2prm_p2 = HsoTerm2aprm_p2 + HsoTerm2bprm_p2 - HsoTerm2cprm_p2
    HsoTerm2coeffprm_p2 = expnu*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**2)
    HsoTerm1prm_p2 = SdotSkerrhat*exp2nu*pdotxirprm_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhatprm_p2*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - Qprm_p2*SdotSkerrhat*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq)
    Hsoprm_p2 = HsoTerm1prm_p2 + HsoTerm2*HsoTerm2coeffprm_p2 + HsoTerm2coeff*HsoTerm2prm_p2
    Hdprm_p2 = Hdcoeff*Hdsumprm_p2
    Hnsprm_p2 = betapsumprm_p2 + Hnsradicandprm_p2*alpha/(2*np.sqrt(Hnsradicand))
    Hsprm_p2 = Hsoprm_p2 + Hssprm_p2
    Heffprm_p2 = -Hdprm_p2 + Hnsprm_p2 + Hsprm_p2
    prT_xprm_p2 = csi2*n2_x + csi2_x*n2
    phat3_xprm_p2 = n3*prT_xprm_p2*(1 - 1/csi1) + n3_x*prTprm_p2*(1 - 1/csi1) + csi1_x*n3*prTprm_p2/csi1**2
    phat2_xprm_p2 = n2*prT_xprm_p2*(1 - 1/csi1) + n2_x*prTprm_p2*(1 - 1/csi1) + csi1_x*n2*prTprm_p2/csi1**2
    phat1_xprm_p2 = n1*prT_xprm_p2*(1 - 1/csi1) + n1_x*prTprm_p2*(1 - 1/csi1) + csi1_x*n1*prTprm_p2/csi1**2
    pdotxir_xprm_p2 = r*(phat1_xprm_p2*xi1 + phat1prm_p2*xi1_x + phat2_xprm_p2*xi2 + phat2prm_p2*xi2_x + phat3_xprm_p2*xi3 + phat3prm_p2*xi3_x) + r_x*(phat1prm_p2*xi1 + phat2prm_p2*xi2 + phat3prm_p2*xi3)
    pdotn_xprm_p2 = n1*phat1_xprm_p2 + n1_x*phat1prm_p2 + n2*phat2_xprm_p2 + n2_x*phat2prm_p2 + n3*phat3_xprm_p2 + n3_x*phat3prm_p2
    pdotvr_xprm_p2 = r*(phat1_xprm_p2*v1 + phat1prm_p2*v1_x + phat2_xprm_p2*v2 + phat2prm_p2*v2_x + phat3_xprm_p2*v3 + phat3prm_p2*v3_x) + r_x*(phat1prm_p2*v1 + phat2prm_p2*v2 + phat3prm_p2*v3)
    pphi_xprm_p2 = pdotxir_xprm_p2
    DrSipn2_xprm_p2 = 2*Deltar*pdotn*pdotn_xprm_p2/Sigma + 2*Deltar*pdotn_x*pdotnprm_p2/Sigma - 2*Deltar*Sigma_x*pdotn*pdotnprm_p2/Sigma**2 + 2*Deltar_x*pdotn*pdotnprm_p2/Sigma
    Q_xprm_p2 = DrSipn2_xprm_p2 + 2*Qcoeff1*pdotxir*pdotxir_xprm_p2 + 2*Qcoeff1*pdotxir_x*pdotxirprm_p2 + 2*Qcoeff1_x*pdotxir*pdotxirprm_p2 + 2*Qcoeff2*pdotvr*pdotvr_xprm_p2 + 2*Qcoeff2*pdotvr_x*pdotvrprm_p2 + 2*Qcoeff2_x*pdotvr*pdotvrprm_p2
    Qminus1_xprm_p2 = Q_xprm_p2
    sigmacoeffTerm2_xprm_p2 = eta*(eta*(r*(-882*DrSipn2_xprm_p2 + 204*Qminus1_xprm_p2 + r*(1620*DrSipn2*DrSipn2_xprm_p2 - 234*DrSipn2*Qminus1_xprm_p2 + 1620*DrSipn2_x*DrSipn2prm_p2 - 234*DrSipn2_x*Qminus1prm_p2 - 234*DrSipn2_xprm_p2*Qminus1 - 234*DrSipn2prm_p2*Qminus1_x) + r_x*(1620*DrSipn2*DrSipn2prm_p2 - 234*DrSipn2*Qminus1prm_p2 - 234*DrSipn2prm_p2*Qminus1)) + r_x*(-882*DrSipn2prm_p2 + 204*Qminus1prm_p2 + r*(1620*DrSipn2*DrSipn2prm_p2 - 234*DrSipn2*Qminus1prm_p2 - 234*DrSipn2prm_p2*Qminus1))) + r*(-96*DrSipn2_xprm_p2 - 436*Qminus1_xprm_p2 + r*(36*DrSipn2*Qminus1_xprm_p2 + 36*DrSipn2_x*Qminus1prm_p2 + 36*DrSipn2_xprm_p2*Qminus1 + 36*DrSipn2prm_p2*Qminus1_x - 90*Qminus1*Qminus1_xprm_p2 - 90*Qminus1_x*Qminus1prm_p2) + r_x*(36*DrSipn2*Qminus1prm_p2 + 36*DrSipn2prm_p2*Qminus1 - 90*Qminus1*Qminus1prm_p2)) + r_x*(-96*DrSipn2prm_p2 - 436*Qminus1prm_p2 + r*(36*DrSipn2*Qminus1prm_p2 + 36*DrSipn2prm_p2*Qminus1 - 90*Qminus1*Qminus1prm_p2)))/(144*r**2) - eta*r_x*(eta*r*(-882*DrSipn2prm_p2 + 204*Qminus1prm_p2 + r*(1620*DrSipn2*DrSipn2prm_p2 - 234*DrSipn2*Qminus1prm_p2 - 234*DrSipn2prm_p2*Qminus1)) + r*(-96*DrSipn2prm_p2 - 436*Qminus1prm_p2 + r*(36*DrSipn2*Qminus1prm_p2 + 36*DrSipn2prm_p2*Qminus1 - 90*Qminus1*Qminus1prm_p2)))/(72*r**3)
    sigmacoeffTerm1_xprm_p2 = eta*(-36*DrSipn2_xprm_p2 + 3*Qminus1_xprm_p2)/12
    sigmacoeff_xprm_p2 = sigmacoeffTerm1_xprm_p2 + sigmacoeffTerm2_xprm_p2
    sigmastarcoeffTerm2_xprm_p2 = eta*(eta*(r*(-324*DrSipn2_xprm_p2 + 120*Qminus1_xprm_p2 + r*(720*DrSipn2*DrSipn2_xprm_p2 + 720*DrSipn2_x*DrSipn2prm_p2 + Qminus1*(-126*DrSipn2_xprm_p2 - 3*Qminus1_xprm_p2) + Qminus1_x*(-126*DrSipn2prm_p2 - 3*Qminus1prm_p2) + Qminus1_xprm_p2*(-126*DrSipn2 - 3*Qminus1) + Qminus1prm_p2*(-126*DrSipn2_x - 3*Qminus1_x)) + r_x*(720*DrSipn2*DrSipn2prm_p2 + Qminus1*(-126*DrSipn2prm_p2 - 3*Qminus1prm_p2) + Qminus1prm_p2*(-126*DrSipn2 - 3*Qminus1))) + r_x*(-324*DrSipn2prm_p2 + 120*Qminus1prm_p2 + r*(720*DrSipn2*DrSipn2prm_p2 + Qminus1*(-126*DrSipn2prm_p2 - 3*Qminus1prm_p2) + Qminus1prm_p2*(-126*DrSipn2 - 3*Qminus1)))) + r*(282*DrSipn2_xprm_p2 + Qminus1*r*(96*DrSipn2_xprm_p2 - 23*Qminus1_xprm_p2) + Qminus1*r_x*(96*DrSipn2prm_p2 - 23*Qminus1prm_p2) + Qminus1_x*r*(96*DrSipn2prm_p2 - 23*Qminus1prm_p2) + Qminus1_xprm_p2*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1_xprm_p2 + Qminus1prm_p2*r*(96*DrSipn2_x - 23*Qminus1_x) + Qminus1prm_p2*r_x*(96*DrSipn2 - 23*Qminus1)) + r_x*(282*DrSipn2prm_p2 + Qminus1*r*(96*DrSipn2prm_p2 - 23*Qminus1prm_p2) + Qminus1prm_p2*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1prm_p2))/(72*r**2) - eta*r_x*(eta*r*(-324*DrSipn2prm_p2 + 120*Qminus1prm_p2 + r*(720*DrSipn2*DrSipn2prm_p2 + Qminus1*(-126*DrSipn2prm_p2 - 3*Qminus1prm_p2) + Qminus1prm_p2*(-126*DrSipn2 - 3*Qminus1))) + r*(282*DrSipn2prm_p2 + Qminus1*r*(96*DrSipn2prm_p2 - 23*Qminus1prm_p2) + Qminus1prm_p2*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1prm_p2))/(36*r**3)
    sigmastarcoeffTerm1_xprm_p2 = eta*(-30*DrSipn2_xprm_p2 + 4*Qminus1_xprm_p2)/12
    sigmastarcoeff_xprm_p2 = sigmastarcoeffTerm1_xprm_p2 + sigmastarcoeffTerm2_xprm_p2
    Deltasigmastar3_xprm_p2 = sigma3*sigmacoeff_xprm_p2 + sigmastar3*sigmastarcoeff_xprm_p2
    Deltasigmastar2_xprm_p2 = sigma2*sigmacoeff_xprm_p2 + sigmastar2*sigmastarcoeff_xprm_p2
    Deltasigmastar1_xprm_p2 = sigma1*sigmacoeff_xprm_p2 + sigmastar1*sigmastarcoeff_xprm_p2
    Sstar3_xprm_p2 = Deltasigmastar3_xprm_p2
    Sstar2_xprm_p2 = Deltasigmastar2_xprm_p2
    Sstar1_xprm_p2 = Deltasigmastar1_xprm_p2
    S3_xprm_p2 = Sstar3_xprm_p2
    S2_xprm_p2 = Sstar2_xprm_p2
    S1_xprm_p2 = Sstar1_xprm_p2
    Sstardotn_xprm_p2 = Sstar1_xprm_p2*n1 + Sstar1prm_p2*n1_x + Sstar2_xprm_p2*n2 + Sstar2prm_p2*n2_x + Sstar3_xprm_p2*n3 + Sstar3prm_p2*n3_x
    SdotSkerrhat_xprm_p2 = S1_xprm_p2*Skerrhat1 + S2_xprm_p2*Skerrhat2 + S3_xprm_p2*Skerrhat3
    Sdotn_xprm_p2 = S1_xprm_p2*n1 + S1prm_p2*n1_x + S2_xprm_p2*n2 + S2prm_p2*n2_x + S3_xprm_p2*n3 + S3prm_p2*n3_x
    Sdotv_xprm_p2 = S1_xprm_p2*v1 + S1prm_p2*v1_x + S2_xprm_p2*v2 + S2prm_p2*v2_x + S3_xprm_p2*v3 + S3prm_p2*v3_x
    Sdotxi_xprm_p2 = S1_xprm_p2*xi1 + S1prm_p2*xi1_x + S2_xprm_p2*xi2 + S2prm_p2*xi2_x + S3_xprm_p2*xi3 + S3prm_p2*xi3_x
    HdsumTerm2_xprm_p2 = 6*Sstardotn*Sstardotn_xprm_p2 + 6*Sstardotn_x*Sstardotnprm_p2
    HdsumTerm1_xprm_p2 = 2*Sstar1*Sstar1_xprm_p2 + 2*Sstar1_x*Sstar1prm_p2 + 2*Sstar2*Sstar2_xprm_p2 + 2*Sstar2_x*Sstar2prm_p2 + 2*Sstar3*Sstar3_xprm_p2 + 2*Sstar3_x*Sstar3prm_p2
    Hdsum_xprm_p2 = HdsumTerm1_xprm_p2 - HdsumTerm2_xprm_p2
    Q4_xprm_p2 = 8*eta*prT**3*prT_xprm_p2*u**2*(-3*eta + 4) + 16*eta*prT**3*prTprm_p2*u*u_x*(-3*eta + 4) + 24*eta*prT**2*prT_x*prTprm_p2*u**2*(-3*eta + 4)
    gammappsum_xprm_p2 = 2*Deltar*pdotn*pdotn_xprm_p2/Sigma + 2*Deltar*pdotn_x*pdotnprm_p2/Sigma - 2*Deltar*Sigma_x*pdotn*pdotnprm_p2/Sigma**2 + 2*Deltar_x*pdotn*pdotnprm_p2/Sigma + 2*pdotvr*pdotvr_xprm_p2/(Sigma*sin2theta) - 2*pdotvr*pdotvrprm_p2*sin2theta_x/(Sigma*sin2theta**2) + 2*pdotvr_x*pdotvrprm_p2/(Sigma*sin2theta) - 2*Sigma_x*pdotvr*pdotvrprm_p2/(Sigma**2*sin2theta) + 2*Sigma*pdotxir*pdotxir_xprm_p2/(Lambt*sin2theta) - 2*Sigma*pdotxir*pdotxirprm_p2*sin2theta_x/(Lambt*sin2theta**2) + 2*Sigma*pdotxir_x*pdotxirprm_p2/(Lambt*sin2theta) + 2*Sigma_x*pdotxir*pdotxirprm_p2/(Lambt*sin2theta) - 2*Lambt_x*Sigma*pdotxir*pdotxirprm_p2/(Lambt**2*sin2theta)
    Hnsradicand_xprm_p2 = Q4_xprm_p2 + gammappsum_xprm_p2
    betapsum_xprm_p2 = omegatilde*pphi_xprm_p2/Lambt + omegatilde_x*pphiprm_p2/Lambt - Lambt_x*omegatilde*pphiprm_p2/Lambt**2
    HssTerm3_xprm_p2 = Btilde**2*(Sdotn*exp2mu*xisq*(-Q_xprm_p2 - Q_xprm_p2/(2*np.sqrt(Q)) + Q_x*Qprm_p2/(4*Q**(3/2))) + Sdotn*exp2mu*xisq_x*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q))) + Sdotn*exp2mu_x*xisq*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q))) + Sdotn_x*exp2mu*xisq*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q))) + Sdotn_xprm_p2*exp2mu*xisq*(-np.sqrt(Q) - Q) + Sdotnprm_p2*exp2mu*xisq*(-Q_x - Q_x/(2*np.sqrt(Q))) + Sdotnprm_p2*exp2mu*xisq_x*(-np.sqrt(Q) - Q) + Sdotnprm_p2*exp2mu_x*xisq*(-np.sqrt(Q) - Q) + pdotvr*(-Jtilde*Sdotv*pdotn_xprm_p2 - Jtilde*Sdotv_x*pdotnprm_p2 - Jtilde*Sdotv_xprm_p2*pdotn - Jtilde*Sdotvprm_p2*pdotn_x - Jtilde_x*Sdotv*pdotnprm_p2 - Jtilde_x*Sdotvprm_p2*pdotn + Sdotn*pdotvr_xprm_p2 + Sdotn_x*pdotvrprm_p2 + Sdotn_xprm_p2*pdotvr + Sdotnprm_p2*pdotvr_x) + pdotvr_x*(-Jtilde*Sdotv*pdotnprm_p2 - Jtilde*Sdotvprm_p2*pdotn + Sdotn*pdotvrprm_p2 + Sdotnprm_p2*pdotvr) + pdotvr_xprm_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + pdotvrprm_p2*(-Jtilde*Sdotv*pdotn_x - Jtilde*Sdotv_x*pdotn - Jtilde_x*Sdotv*pdotn + Sdotn*pdotvr_x + Sdotn_x*pdotvr)) + Btilde*Btilde_x*(Sdotn*exp2mu*xisq*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q)) + Sdotnprm_p2*exp2mu*xisq*(-2*np.sqrt(Q) - 2*Q) + pdotvr*(-2*Jtilde*Sdotv*pdotnprm_p2 - 2*Jtilde*Sdotvprm_p2*pdotn + 2*Sdotn*pdotvrprm_p2 + 2*Sdotnprm_p2*pdotvr) + pdotvrprm_p2*(-2*Jtilde*Sdotv*pdotn + 2*Sdotn*pdotvr)) + expmu*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn_xprm_p2 + Btilde*Jtilde*Sdotxi_x*pdotnprm_p2 + Btilde*Jtilde*Sdotxi_xprm_p2*pdotn + Btilde*Jtilde*Sdotxiprm_p2*pdotn_x + Btilde*Jtilde_x*Sdotxi*pdotnprm_p2 + Btilde*Jtilde_x*Sdotxiprm_p2*pdotn + Btilde_x*Jtilde*Sdotxi*pdotnprm_p2 + Btilde_x*Jtilde*Sdotxiprm_p2*pdotn - Sdotn*expmu*expnu*pdotxir_xprm_p2 - Sdotn*expmu*expnu_x*pdotxirprm_p2 - Sdotn*expmu_x*expnu*pdotxirprm_p2 - Sdotn_x*expmu*expnu*pdotxirprm_p2 - Sdotn_xprm_p2*expmu*expnu*pdotxir - Sdotnprm_p2*expmu*expnu*pdotxir_x - Sdotnprm_p2*expmu*expnu_x*pdotxir - Sdotnprm_p2*expmu_x*expnu*pdotxir) + expmu*expnu*pdotxir_x*(Btilde*Jtilde*Sdotxi*pdotnprm_p2 + Btilde*Jtilde*Sdotxiprm_p2*pdotn - Sdotn*expmu*expnu*pdotxirprm_p2 - Sdotnprm_p2*expmu*expnu*pdotxir) + expmu*expnu*pdotxir_xprm_p2*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir) + expmu*expnu*pdotxirprm_p2*(Btilde*Jtilde*Sdotxi*pdotn_x + Btilde*Jtilde*Sdotxi_x*pdotn + Btilde*Jtilde_x*Sdotxi*pdotn + Btilde_x*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir_x - Sdotn*expmu*expnu_x*pdotxir - Sdotn*expmu_x*expnu*pdotxir - Sdotn_x*expmu*expnu*pdotxir) + expmu*expnu_x*pdotxir*(Btilde*Jtilde*Sdotxi*pdotnprm_p2 + Btilde*Jtilde*Sdotxiprm_p2*pdotn - Sdotn*expmu*expnu*pdotxirprm_p2 - Sdotnprm_p2*expmu*expnu*pdotxir) + expmu*expnu_x*pdotxirprm_p2*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir) + expmu_x*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotnprm_p2 + Btilde*Jtilde*Sdotxiprm_p2*pdotn - Sdotn*expmu*expnu*pdotxirprm_p2 - Sdotnprm_p2*expmu*expnu*pdotxir) + expmu_x*expnu*pdotxirprm_p2*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir)
    HssTerm3coeff_xprm_p2 = omegacostheta*(-Q_xprm_p2 - Q_xprm_p2/(2*np.sqrt(Q)) + Q_x*Qprm_p2/(4*Q**(3/2)))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**2) + omegacostheta*(-Q_x - Q_x/(2*np.sqrt(Q)))*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**3) + omegacostheta_x*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu*expnu*(2*np.sqrt(Q) + 2*Q)**2) - expnu_x*omegacostheta*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu*expnu**2*(2*np.sqrt(Q) + 2*Q)**2) - expmu_x*omegacostheta*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu**2*expnu*(2*np.sqrt(Q) + 2*Q)**2) - exp2mu_x*omegacostheta*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu**2*expmu*expnu*(2*np.sqrt(Q) + 2*Q)**2) - Btilde_x*omegacostheta*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde**2*exp2mu*expmu*expnu*(2*np.sqrt(Q) + 2*Q)**2)
    HssTerm2_xprm_p2 = Btilde**2*xisq*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotn_xprm_p2 - Jtilde*Sdotv_x*pdotnprm_p2 - Jtilde*Sdotv_xprm_p2*pdotn - Jtilde*Sdotvprm_p2*pdotn_x - Jtilde_x*Sdotv*pdotnprm_p2 - Jtilde_x*Sdotvprm_p2*pdotn + Sdotn*pdotvr_xprm_p2 + Sdotn_x*pdotvrprm_p2 + Sdotn_xprm_p2*pdotvr + Sdotnprm_p2*pdotvr_x) + Jtilde*pdotn_x*(-Jtilde*Sdotv*pdotnprm_p2 - Jtilde*Sdotvprm_p2*pdotn + Sdotn*pdotvrprm_p2 + Sdotnprm_p2*pdotvr) + Jtilde*pdotn_xprm_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Jtilde*pdotnprm_p2*(-Jtilde*Sdotv*pdotn_x - Jtilde*Sdotv_x*pdotn - Jtilde_x*Sdotv*pdotn + Sdotn*pdotvr_x + Sdotn_x*pdotvr) + Jtilde_x*pdotn*(-Jtilde*Sdotv*pdotnprm_p2 - Jtilde*Sdotvprm_p2*pdotn + Sdotn*pdotvrprm_p2 + Sdotnprm_p2*pdotvr) + Jtilde_x*pdotnprm_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Sdotv*exp2mu*(Q_xprm_p2 + Q_xprm_p2/(2*np.sqrt(Q)) - Q_x*Qprm_p2/(4*Q**(3/2))) + Sdotv*exp2mu_x*(Qprm_p2 + Qprm_p2/(2*np.sqrt(Q))) + Sdotv_x*exp2mu*(Qprm_p2 + Qprm_p2/(2*np.sqrt(Q))) + Sdotv_xprm_p2*exp2mu*(np.sqrt(Q) + Q) + Sdotvprm_p2*exp2mu*(Q_x + Q_x/(2*np.sqrt(Q))) + Sdotvprm_p2*exp2mu_x*(np.sqrt(Q) + Q)) + Btilde**2*xisq_x*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotnprm_p2 - Jtilde*Sdotvprm_p2*pdotn + Sdotn*pdotvrprm_p2 + Sdotnprm_p2*pdotvr) + Jtilde*pdotnprm_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Sdotv*exp2mu*(Qprm_p2 + Qprm_p2/(2*np.sqrt(Q))) + Sdotvprm_p2*exp2mu*(np.sqrt(Q) + Q)) + Btilde*Btilde_x*xisq*(Jtilde*pdotn*(-2*Jtilde*Sdotv*pdotnprm_p2 - 2*Jtilde*Sdotvprm_p2*pdotn + 2*Sdotn*pdotvrprm_p2 + 2*Sdotnprm_p2*pdotvr) + Jtilde*pdotnprm_p2*(-2*Jtilde*Sdotv*pdotn + 2*Sdotn*pdotvr) + Sdotv*exp2mu*(2*Qprm_p2 + Qprm_p2/np.sqrt(Q)) + Sdotvprm_p2*exp2mu*(2*np.sqrt(Q) + 2*Q)) + expmu*pdotxir*(-Btilde*Sdotxi*expnu*pdotvr_xprm_p2 - Btilde*Sdotxi*expnu_x*pdotvrprm_p2 - Btilde*Sdotxi_x*expnu*pdotvrprm_p2 - Btilde*Sdotxi_xprm_p2*expnu*pdotvr - Btilde*Sdotxiprm_p2*expnu*pdotvr_x - Btilde*Sdotxiprm_p2*expnu_x*pdotvr - Btilde_x*Sdotxi*expnu*pdotvrprm_p2 - Btilde_x*Sdotxiprm_p2*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir_xprm_p2 + Sdotv*exp2nu*expmu_x*pdotxirprm_p2 + Sdotv*exp2nu_x*expmu*pdotxirprm_p2 + Sdotv_x*exp2nu*expmu*pdotxirprm_p2 + Sdotv_xprm_p2*exp2nu*expmu*pdotxir + Sdotvprm_p2*exp2nu*expmu*pdotxir_x + Sdotvprm_p2*exp2nu*expmu_x*pdotxir + Sdotvprm_p2*exp2nu_x*expmu*pdotxir) + expmu*pdotxir_x*(-Btilde*Sdotxi*expnu*pdotvrprm_p2 - Btilde*Sdotxiprm_p2*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxirprm_p2 + Sdotvprm_p2*exp2nu*expmu*pdotxir) + expmu*pdotxir_xprm_p2*(-Btilde*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir) + expmu*pdotxirprm_p2*(-Btilde*Sdotxi*expnu*pdotvr_x - Btilde*Sdotxi*expnu_x*pdotvr - Btilde*Sdotxi_x*expnu*pdotvr - Btilde_x*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir_x + Sdotv*exp2nu*expmu_x*pdotxir + Sdotv*exp2nu_x*expmu*pdotxir + Sdotv_x*exp2nu*expmu*pdotxir) + expmu_x*pdotxir*(-Btilde*Sdotxi*expnu*pdotvrprm_p2 - Btilde*Sdotxiprm_p2*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxirprm_p2 + Sdotvprm_p2*exp2nu*expmu*pdotxir) + expmu_x*pdotxirprm_p2*(-Btilde*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir)
    HssTerm2coeff_xprm_p2 = Jtilde*omegar*(-Q_xprm_p2 - Q_xprm_p2/(2*np.sqrt(Q)) + Q_x*Qprm_p2/(4*Q**(3/2)))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**2) + Jtilde*omegar*(-Q_x - Q_x/(2*np.sqrt(Q)))*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**3) - Jtilde*omegar*xisq_x*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu*expnu*xisq**2*(2*np.sqrt(Q) + 2*Q)**2) + Jtilde*omegar_x*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q) + 2*Q)**2) - Jtilde*expnu_x*omegar*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu*expnu**2*xisq*(2*np.sqrt(Q) + 2*Q)**2) - Jtilde*expmu_x*omegar*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu**2*expnu*xisq*(2*np.sqrt(Q) + 2*Q)**2) - Jtilde*exp2mu_x*omegar*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu**2*expmu*expnu*xisq*(2*np.sqrt(Q) + 2*Q)**2) + Jtilde_x*omegar*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q) + 2*Q)**2) - Btilde_x*Jtilde*omegar*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde**2*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q) + 2*Q)**2)
    HssTerm1_xprm_p2 = SdotSkerrhat_xprm_p2*omega + SdotSkerrhatprm_p2*omega_x
    Hss_xprm_p2 = HssTerm1_xprm_p2 + HssTerm2*HssTerm2coeff_xprm_p2 + HssTerm2_x*HssTerm2coeffprm_p2 + HssTerm2_xprm_p2*HssTerm2coeff + HssTerm2coeff_x*HssTerm2prm_p2 + HssTerm3*HssTerm3coeff_xprm_p2 + HssTerm3_x*HssTerm3coeffprm_p2 + HssTerm3_xprm_p2*HssTerm3coeff + HssTerm3coeff_x*HssTerm3prm_p2
    HsoTerm2c_xprm_p2 = Brtilde*Jtilde*Sdotv*expmu*expnu*pdotxir_xprm_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv*expmu*expnu_x*pdotxirprm_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv*expmu_x*expnu*pdotxirprm_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv_x*expmu*expnu*pdotxirprm_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv_xprm_p2*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotvprm_p2*expmu*expnu*pdotxir_x*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotvprm_p2*expmu*expnu_x*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotvprm_p2*expmu_x*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Q_x*Sdotv*expmu*expnu*pdotxirprm_p2/(2*np.sqrt(Q)) + Brtilde*Jtilde*Q_x*Sdotvprm_p2*expmu*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Q_xprm_p2*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Qprm_p2*Sdotv*expmu*expnu*pdotxir_x/(2*np.sqrt(Q)) + Brtilde*Jtilde*Qprm_p2*Sdotv*expmu*expnu_x*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Qprm_p2*Sdotv*expmu_x*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Qprm_p2*Sdotv_x*expmu*expnu*pdotxir/(2*np.sqrt(Q)) - Brtilde*Jtilde*Q_x*Qprm_p2*Sdotv*expmu*expnu*pdotxir/(4*Q**(3/2)) + Brtilde*Jtilde_x*Sdotv*expmu*expnu*pdotxirprm_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde_x*Sdotvprm_p2*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde_x*Qprm_p2*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde_x*Jtilde*Sdotv*expmu*expnu*pdotxirprm_p2*(np.sqrt(Q) + 1) + Brtilde_x*Jtilde*Sdotvprm_p2*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde_x*Jtilde*Qprm_p2*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q))
    HsoTerm2b_xprm_p2 = Btilde*expmu*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv_xprm_p2*nur + Jtilde*Sdotvprm_p2*nur_x + Jtilde_x*Sdotvprm_p2*nur - Sdotn_xprm_p2*nucostheta*xisq - Sdotnprm_p2*nucostheta*xisq_x - Sdotnprm_p2*nucostheta_x*xisq) + Btilde*expmu*expnu*pdotxir_x*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotvprm_p2*nur - Sdotnprm_p2*nucostheta*xisq) + Btilde*expmu*expnu*pdotxir_xprm_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*expmu*expnu*pdotxirprm_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur_x + Jtilde*Sdotv_x*nur + Jtilde_x*Sdotv*nur - Sdotn*nucostheta*xisq_x - Sdotn*nucostheta_x*xisq - Sdotn_x*nucostheta*xisq) + Btilde*expmu*expnu_x*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotvprm_p2*nur - Sdotnprm_p2*nucostheta*xisq) + Btilde*expmu*expnu_x*pdotxirprm_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*expmu_x*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotvprm_p2*nur - Sdotnprm_p2*nucostheta*xisq) + Btilde*expmu_x*expnu*pdotxirprm_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*Q_x*expmu*expnu*pdotxir*(Jtilde*Sdotvprm_p2*nur - Sdotnprm_p2*nucostheta*xisq)/np.sqrt(Q) + Btilde*Q_x*expmu*expnu*pdotxirprm_p2*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Q_xprm_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Qprm_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur_x + Jtilde*Sdotv_x*nur + Jtilde_x*Sdotv*nur - Sdotn*nucostheta*xisq_x - Sdotn*nucostheta_x*xisq - Sdotn_x*nucostheta*xisq)/np.sqrt(Q) + Btilde*Qprm_p2*expmu*expnu*pdotxir_x*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Qprm_p2*expmu*expnu_x*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Qprm_p2*expmu_x*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) - Btilde*Q_x*Qprm_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/(2*Q**(3/2)) + Btilde_x*expmu*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotvprm_p2*nur - Sdotnprm_p2*nucostheta*xisq) + Btilde_x*expmu*expnu*pdotxirprm_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde_x*Qprm_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q)
    HsoTerm2a_xprm_p2 = Btilde**2*Jtilde*Sdotxi*(-np.sqrt(Q)*(nur*pdotvr_xprm_p2 + nur_x*pdotvrprm_p2 + pdotn_xprm_p2*xisq*(mucostheta - nucostheta) + pdotnprm_p2*xisq*(mucostheta_x - nucostheta_x) + pdotnprm_p2*xisq_x*(mucostheta - nucostheta)) - mucostheta*pdotn_xprm_p2*xisq - mucostheta*pdotnprm_p2*xisq_x - mucostheta_x*pdotnprm_p2*xisq + mur*pdotvr_xprm_p2*(np.sqrt(Q) + 1) + mur_x*pdotvrprm_p2*(np.sqrt(Q) + 1) + Q_x*mur*pdotvrprm_p2/(2*np.sqrt(Q)) + Q_x*(-nur*pdotvrprm_p2 + pdotnprm_p2*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q)) + Q_xprm_p2*mur*pdotvr/(2*np.sqrt(Q)) + Q_xprm_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q)) + Qprm_p2*mur*pdotvr_x/(2*np.sqrt(Q)) + Qprm_p2*mur_x*pdotvr/(2*np.sqrt(Q)) - Qprm_p2*(nur*pdotvr_x + nur_x*pdotvr + pdotn*xisq*(mucostheta_x - nucostheta_x) + pdotn*xisq_x*(mucostheta - nucostheta) + pdotn_x*xisq*(mucostheta - nucostheta))/(2*np.sqrt(Q)) - Q_x*Qprm_p2*mur*pdotvr/(4*Q**(3/2)) - Q_x*Qprm_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(4*Q**(3/2))) + Btilde**2*Jtilde*Sdotxi_x*(np.sqrt(Q)*(-nur*pdotvrprm_p2 + pdotnprm_p2*xisq*(-mucostheta + nucostheta)) - mucostheta*pdotnprm_p2*xisq + mur*pdotvrprm_p2*(np.sqrt(Q) + 1) + Qprm_p2*mur*pdotvr/(2*np.sqrt(Q)) + Qprm_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtilde*Sdotxi_xprm_p2*(np.sqrt(Q)*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta)) - mucostheta*pdotn*xisq + mur*pdotvr*(np.sqrt(Q) + 1)) + Btilde**2*Jtilde*Sdotxiprm_p2*(-np.sqrt(Q)*(nur*pdotvr_x + nur_x*pdotvr + pdotn*xisq*(mucostheta_x - nucostheta_x) + pdotn*xisq_x*(mucostheta - nucostheta) + pdotn_x*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn*xisq_x - mucostheta*pdotn_x*xisq - mucostheta_x*pdotn*xisq + mur*pdotvr_x*(np.sqrt(Q) + 1) + mur_x*pdotvr*(np.sqrt(Q) + 1) + Q_x*mur*pdotvr/(2*np.sqrt(Q)) + Q_x*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtilde_x*Sdotxi*(np.sqrt(Q)*(-nur*pdotvrprm_p2 + pdotnprm_p2*xisq*(-mucostheta + nucostheta)) - mucostheta*pdotnprm_p2*xisq + mur*pdotvrprm_p2*(np.sqrt(Q) + 1) + Qprm_p2*mur*pdotvr/(2*np.sqrt(Q)) + Qprm_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtilde_x*Sdotxiprm_p2*(np.sqrt(Q)*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta)) - mucostheta*pdotn*xisq + mur*pdotvr*(np.sqrt(Q) + 1)) + Btilde*Btilde_x*Jtilde*Sdotxi*(np.sqrt(Q)*(-2*nur*pdotvrprm_p2 + 2*pdotnprm_p2*xisq*(-mucostheta + nucostheta)) - 2*mucostheta*pdotnprm_p2*xisq + mur*pdotvrprm_p2*(2*np.sqrt(Q) + 2) + Qprm_p2*mur*pdotvr/np.sqrt(Q) + Qprm_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/np.sqrt(Q)) + Btilde*Btilde_x*Jtilde*Sdotxiprm_p2*(np.sqrt(Q)*(-2*nur*pdotvr + 2*pdotn*xisq*(-mucostheta + nucostheta)) - 2*mucostheta*pdotn*xisq + mur*pdotvr*(2*np.sqrt(Q) + 2))
    HsoTerm2_xprm_p2 = HsoTerm2a_xprm_p2 + HsoTerm2b_xprm_p2 - HsoTerm2c_xprm_p2
    HsoTerm2coeff_xprm_p2 = expnu*(-Q_xprm_p2 - Q_xprm_p2/(2*np.sqrt(Q)) + Q_x*Qprm_p2/(4*Q**(3/2)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**2) + expnu*(-Q_x - Q_x/(2*np.sqrt(Q)))*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**3) - expnu*xisq_x*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq**2*(np.sqrt(Q) + Q)**2) + expnu_x*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**2) - exp2mu_x*expnu*(-Qprm_p2 - Qprm_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu**2*xisq*(np.sqrt(Q) + Q)**2) - Btilde_x*expnu*(-2*Qprm_p2 - Qprm_p2/np.sqrt(Q))/(Btilde**3*exp2mu*xisq*(np.sqrt(Q) + Q)**2)
    HsoTerm1_xprm_p2 = SdotSkerrhat*exp2nu*pdotxir_xprm_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat*exp2nu*pdotxirprm_p2*(-Btilde_x + expmu*expnu_x + expmu_x*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat*exp2nu*pdotxirprm_p2*xisq_x*(Btilde - expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq**2) - SdotSkerrhat*exp2nu*expmu_x*pdotxirprm_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu**2*xisq) + SdotSkerrhat*exp2nu_x*pdotxirprm_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat_x*exp2nu*pdotxirprm_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat_xprm_p2*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhatprm_p2*exp2nu*pdotxir*(-Btilde_x + expmu*expnu_x + expmu_x*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhatprm_p2*exp2nu*pdotxir*xisq_x*(Btilde - expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq**2) + SdotSkerrhatprm_p2*exp2nu*pdotxir_x*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - SdotSkerrhatprm_p2*exp2nu*expmu_x*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu**2*xisq) + SdotSkerrhatprm_p2*exp2nu_x*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - Q_x*SdotSkerrhat*exp2nu*pdotxirprm_p2*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Q_x*SdotSkerrhatprm_p2*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Q_xprm_p2*SdotSkerrhat*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Qprm_p2*SdotSkerrhat*exp2nu*pdotxir*(-Btilde_x + expmu*expnu_x + expmu_x*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Qprm_p2*SdotSkerrhat*exp2nu*pdotxir*xisq_x*(Btilde - expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq**2) - Qprm_p2*SdotSkerrhat*exp2nu*pdotxir_x*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) + Qprm_p2*SdotSkerrhat*exp2nu*expmu_x*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu**2*xisq) - Qprm_p2*SdotSkerrhat*exp2nu_x*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Qprm_p2*SdotSkerrhat_x*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) + Q_x*Qprm_p2*SdotSkerrhat*exp2nu*pdotxir*(-3*Btilde + 3*expmu*expnu)/(4*Btilde**2*Q**(5/2)*expmu*xisq) - Btilde_x*SdotSkerrhat*exp2nu*pdotxirprm_p2*(-2*Btilde + 2*expmu*expnu)/(Btilde**3*np.sqrt(Q)*expmu*xisq) - Btilde_x*SdotSkerrhatprm_p2*exp2nu*pdotxir*(-2*Btilde + 2*expmu*expnu)/(Btilde**3*np.sqrt(Q)*expmu*xisq) + Btilde_x*Qprm_p2*SdotSkerrhat*exp2nu*pdotxir*(-2*Btilde + 2*expmu*expnu)/(2*Btilde**3*Q**(3/2)*expmu*xisq)
    Hso_xprm_p2 = HsoTerm1_xprm_p2 + HsoTerm2*HsoTerm2coeff_xprm_p2 + HsoTerm2_x*HsoTerm2coeffprm_p2 + HsoTerm2_xprm_p2*HsoTerm2coeff + HsoTerm2coeff_x*HsoTerm2prm_p2
    Hd_xprm_p2 = Hdcoeff*Hdsum_xprm_p2 + Hdcoeff_x*Hdsumprm_p2
    Hns_xprm_p2 = betapsum_xprm_p2 + Hnsradicand_xprm_p2*alpha/(2*np.sqrt(Hnsradicand)) + Hnsradicandprm_p2*alpha_x/(2*np.sqrt(Hnsradicand)) - Hnsradicand_x*Hnsradicandprm_p2*alpha/(4*Hnsradicand**(3/2))
    Hs_xprm_p2 = Hso_xprm_p2 + Hss_xprm_p2
    Heff_xprm_p2 = -Hd_xprm_p2 + Hns_xprm_p2 + Hs_xprm_p2
    Hreal_xprm_p2 = -Heff_x*Heffprm_p2*eta**2/(2*eta*(Heff - 1) + 1)**(3/2) + Heff_xprm_p2*eta/np.sqrt(2*eta*(Heff - 1) + 1)
    return np.array([Hreal_xprm_p2])

## dpy dx

In [3]:
def compute_d2Hdp2dx(m1, m2, eta, x, y, z, p1, p2, p3, S1x, S1y, S1z, S2x, S2y, S2z, KK, k0, k1, dSO, dSS, tortoise, EMgamma):
    M = m1+m2
    mu = m1*m2/M
    eta = mu/M
    r = np.sqrt(x*x+y*y+z*z)
    u = 1/r
    sigmastar3 = m2/m1*S1z+m1/m2*S2z
    sigmastar2 = m2/m1*S1y+m1/m2*S2y
    sigmastar1 = m2/m1*S1x+m1/m2*S2x
    sigma3 = S1z+S2z
    sigma2 = S1y+S2y
    sigma1 = S1x+S2x
    Skerr3 = sigma3
    Skerr2 = sigma2
    Skerr1 = sigma1
    Skerrmag = np.sqrt(Skerr1*Skerr1+Skerr2*Skerr2+Skerr3*Skerr3)
    Skerrhat3 = Skerr3/Skerrmag
    Skerrhat2 = Skerr2/Skerrmag
    Skerrhat1 = Skerr1/Skerrmag
    a = Skerrmag
    n3 = z/r
    n2 = y/r
    n1 = x/r
    e33 = Skerrhat3
    e32 = Skerrhat2
    e31 = Skerrhat1
    xi3 = e31*n2-e32*n1
    xi2 = -e31*n3+e33*n1
    xi1 = e32*n3-e33*n2
    v3 = n1*xi2-n2*xi1
    v2 = n3*xi1-n1*xi3
    v1 = n2*xi3-n3*xi2
    costheta = e31*n1+e32*n2+e33*n3
    sin2theta = 1-costheta*costheta
    xisq = sin2theta
    w2 = a*a+r*r
    Sigma = r*r+a*a*costheta*costheta
    Dinv = 1+np.log(1+6*eta*u*u+2*(26-3*eta)*eta*u*u*u)
    omegatilde = 2*a*r
    K = 1.712-1.803949138004582*eta-39.77229225266885*eta*eta+103.16588921239249*eta*eta*eta
    etaKminus1 = eta*K-1
    Delta0 = K*(eta*K-2)
    Delta1 = -2*etaKminus1*(K+Delta0)
    Delta2 = np.true_divide(1,2)*Delta1*(Delta1-4*etaKminus1)-a*a*etaKminus1*etaKminus1*Delta0
    Delta3 = -np.true_divide(1,3)*Delta1*Delta1*Delta1+etaKminus1*Delta1*Delta1+Delta2*Delta1-2*etaKminus1*(Delta2-etaKminus1)-a*a*etaKminus1*etaKminus1*Delta1
    Delta4 = np.true_divide(1,12)*(6*a*a*(Delta1*Delta1-2*Delta2)*etaKminus1*etaKminus1+3*Delta1*Delta1*Delta1*Delta1-8*etaKminus1*Delta1*Delta1*Delta1-12*Delta2*Delta1*Delta1+12*(2*etaKminus1*Delta2+Delta3)*Delta1+12*(np.true_divide(94,3)-np.true_divide(41,32)*np.pi*np.pi)*etaKminus1*etaKminus1+6*(Delta2*Delta2-4*Delta3*etaKminus1))
    Delta5 = etaKminus1*etaKminus1*((np.true_divide(-4237,60)+np.true_divide(128,5)*EMgamma+np.true_divide(2275,512)*np.pi*np.pi-np.true_divide(1,3)*a*a*(Delta1*Delta1*Delta1-3*Delta1*Delta2+3*Delta3)-(Delta1*Delta1*Delta1*Delta1*Delta1-5*Delta1*Delta1*Delta1*Delta2+5*Delta1*Delta2*Delta2+5*Delta1*Delta1*Delta3-5*Delta2*Delta3-5*Delta1*Delta4)/(5*etaKminus1*etaKminus1)+(Delta1*Delta1*Delta1*Delta1-4*Delta1*Delta1*Delta2+2*Delta2*Delta2+4*Delta1*Delta3-4*Delta4)/(2*etaKminus1)+np.true_divide(256,5)*np.log(2)))
    Delta5l = etaKminus1*etaKminus1*np.true_divide(64,5)
    logarg = u*(Delta1+u*(Delta2+u*(Delta3+u*(Delta4+u*(Delta5+Delta5l*np.log(u))))))
    Deltaucalib = 1+eta*(Delta0+np.log(1+logarg))
    Deltaucalib = -eta*u*u*(Delta1+u*(2*Delta2+u*(3*Delta3+u*(4*Delta4+u*(5*(Delta5+Delta5l*np.log(u)))))))/(1+logarg)
    Deltaubar = a*a*u*u+(2*u+1/etaKminus1)/etaKminus1
    Deltaubar = -2*a*a*u*u*u-2*u*u/(etaKminus1)
    Deltau = Deltaubar*Deltaucalib
    Deltau = Deltaubar*Deltaucalib+Deltaubar*Deltaucalib
    Deltat = 2*r*Deltau+r*r*Deltau
    Deltat = r*r*Deltau
    Deltar = Deltat*Dinv
    Lambt = w2*w2-a*a*Deltat*sin2theta
    csi = np.sqrt(Deltar*Deltat)/w2
    csi1 = 1+(1-abs(1-tortoise))*(csi-1)
    csi2 = 1+(np.true_divide(1,2)-np.true_divide(1,2)*np.sign(np.true_divide(3,2)-tortoise))*(csi-1)
    prT = csi2*(p1*n1+p2*n2+p3*n3)
    phat3 = p3+prT*(1-1/csi1)*n3
    phat2 = p2+prT*(1-1/csi1)*n2
    phat1 = p1+prT*(1-1/csi1)*n1
    pdotxir = (phat1*xi1+phat2*xi2+phat3*xi3)*r
    pdotn = phat1*n1+phat2*n2+phat3*n3
    pdotvr = (phat1*v1+phat2*v2+phat3*v3)*r
    pphi = pdotxir
    Qcoeff2 = 1/(Sigma*sin2theta)
    Qcoeff1 = Sigma/(Lambt*sin2theta)
    DrSipn2 = Deltar*pdotn*pdotn/Sigma
    Q = 1+DrSipn2+Qcoeff1*pdotxir*pdotxir+Qcoeff2*pdotvr*pdotvr
    Qminus1 = Q-1
    Jtilde = np.sqrt(Deltar)
    exp2mu = Sigma
    expmu = np.sqrt(exp2mu)
    Brtilde = (np.sqrt(Deltar)*Deltat-2*Deltat)/(2*np.sqrt(Deltar*Deltat))
    Btilde = np.sqrt(Deltat)
    exp2nu = Deltat*Sigma/Lambt
    expnu = np.sqrt(exp2nu)
    omega = omegatilde/Lambt
    omegatilde = 2*a
    Lambt = 4*(a*a+r*r)*r-2*a*a*Deltat*sin2theta
    mucostheta = a*a*costheta/Sigma
    nucostheta = a*a*w2*costheta*(w2-Deltat)/(Lambt*Sigma)
    omegacostheta = -2*a*a*costheta*Deltat*omegatilde/(Lambt*Lambt)
    mur = r/Sigma-1/np.sqrt(Deltar)
    nur = r/Sigma+w2*(w2*Deltat-4*r*Deltat)/(2*Lambt*Deltat)
    omegar = (Lambt*omegatilde-Lambt*omegatilde)/(Lambt*Lambt)
    dSO = -74.71-156.*eta+627.5*eta*eta
    sigmacoeffTerm3 = eta*dSO*u*u*u
    sigmacoeffTerm2 = eta/(144*r*r)*(-896+r*(-436*Qminus1-96*DrSipn2+r*(-45*Qminus1*Qminus1+36*Qminus1*DrSipn2))+eta*(-336+r*(204*Qminus1-882*DrSipn2+r*(810*DrSipn2*DrSipn2-234*Qminus1*DrSipn2))))
    sigmacoeffTerm1 = eta/12*(-8/r+3*Qminus1-36*DrSipn2)
    sigmacoeff = sigmacoeffTerm1+sigmacoeffTerm2+sigmacoeffTerm3
    sigmastarcoeffTerm2 = eta/(72*r*r)*(706+r*(-206*Qminus1+282*DrSipn2+r*Qminus1*(96*DrSipn2-23*Qminus1))+eta*(-54+r*(120*Qminus1-324*DrSipn2+r*(360*DrSipn2*DrSipn2+Qminus1*(-126*DrSipn2-3*Qminus1)))))
    sigmastarcoeffTerm1 = eta/12*(14/r+4*Qminus1-30*DrSipn2)
    sigmastarcoeff = sigmastarcoeffTerm1+sigmastarcoeffTerm2
    Deltasigmastar3 = sigmastar3*sigmastarcoeff+sigma3*sigmacoeff
    Deltasigmastar2 = sigmastar2*sigmastarcoeff+sigma2*sigmacoeff
    Deltasigmastar1 = sigmastar1*sigmastarcoeff+sigma1*sigmacoeff
    Sstar3 = sigmastar3+Deltasigmastar3
    Sstar2 = sigmastar2+Deltasigmastar2
    Sstar1 = sigmastar1+Deltasigmastar1
    S3 = Sstar3
    S2 = Sstar2
    S1 = Sstar1
    Sstardotn = Sstar1*n1+Sstar2*n2+Sstar3*n3
    SdotSkerrhat = S1*Skerrhat1+S2*Skerrhat2+S3*Skerrhat3
    Sdotn = S1*n1+S2*n2+S3*n3
    Sdotv = S1*v1+S2*v2+S3*v3
    Sdotxi = S1*xi1+S2*xi2+S3*xi3
    HdsumTerm2 = 3*Sstardotn*Sstardotn
    HdsumTerm1 = Sstar1*Sstar1+Sstar2*Sstar2+Sstar3*Sstar3
    Hdsum = HdsumTerm1-HdsumTerm2
    Hdcoeff = np.true_divide(1,2)/(r*r*r)
    Q4 = 2*prT*prT*prT*prT*u*u*(4-3*eta)*eta
    gammappsum = Deltar/Sigma*pdotn*pdotn+1/Sigma*pdotvr*pdotvr/sin2theta+Sigma/Lambt/sin2theta*pdotxir*pdotxir
    Hnsradicand = 1+gammappsum+Q4
    alpha = np.sqrt(Deltat*Sigma/Lambt)
    betapsum = omegatilde*pphi/Lambt
    HssTerm3 = expmu*expnu*pdotxir*(Jtilde*pdotn*Sdotxi*Btilde-expmu*expnu*pdotxir*Sdotn)+(pdotvr*(Sdotn*pdotvr-Jtilde*pdotn*Sdotv)-exp2mu*(np.sqrt(Q)+Q)*Sdotn*xisq)*Btilde*Btilde
    HssTerm3coeff = omegacostheta/(2*exp2mu*expmu*expnu*Btilde*(Q+np.sqrt(Q)))
    HssTerm2 = expmu*pdotxir*(expmu*exp2nu*pdotxir*Sdotv-expnu*pdotvr*Sdotxi*Btilde)+xisq*Btilde*Btilde*(exp2mu*(np.sqrt(Q)+Q)*Sdotv+Jtilde*pdotn*(pdotvr*Sdotn-Jtilde*pdotn*Sdotv))
    HssTerm2coeff = Jtilde*omegar/(2*exp2mu*expmu*expnu*Btilde*(Q+np.sqrt(Q))*xisq)
    HssTerm1 = omega*SdotSkerrhat
    Hss = HssTerm1+HssTerm2coeff*HssTerm2+HssTerm3coeff*HssTerm3
    HsoTerm2c = Jtilde*Brtilde*expmu*expnu*pdotxir*(np.sqrt(Q)+1)*Sdotv
    HsoTerm2b = expmu*expnu*pdotxir*(2*np.sqrt(Q)+1)*(Jtilde*nur*Sdotv-nucostheta*Sdotn*xisq)*Btilde
    HsoTerm2a = Sdotxi*Jtilde*(mur*pdotvr*(np.sqrt(Q)+1)-mucostheta*pdotn*xisq-np.sqrt(Q)*(nur*pdotvr+(mucostheta-nucostheta)*pdotn*xisq))*Btilde*Btilde
    HsoTerm2 = HsoTerm2a+HsoTerm2b-HsoTerm2c
    HsoTerm2coeff = expnu/(exp2mu*Btilde*Btilde*(Q+np.sqrt(Q))*xisq)
    HsoTerm1 = exp2nu*(expmu*expnu-Btilde)*pdotxir*SdotSkerrhat/(expmu*Btilde*Btilde*np.sqrt(Q)*xisq)
    Hso = HsoTerm1+HsoTerm2coeff*HsoTerm2
    Hd = Hdcoeff*Hdsum
    Hns = betapsum+alpha*np.sqrt(Hnsradicand)
    Hs = Hso+Hss
    dSS = 8.127-154.2*eta+830.8*eta*eta
    Heff = Hs+Hns-Hd+dSS*eta*u*u*u*u*(S1x*S1x+S1y*S1y+S1z*S1z+S2x*S2x+S2y*S2y+S2z*S2z)
    prT_p2 = csi2*n2
    phat3_p2 = n3*prT_p2*(1-1/csi1)
    phat2_p2 = n2*prT_p2*(1-1/csi1)+1
    phat1_p2 = n1*prT_p2*(1-1/csi1)
    pdotxir_p2 = r*(phat1_p2*xi1+phat2_p2*xi2+phat3_p2*xi3)
    pdotn_p2 = n1*phat1_p2+n2*phat2_p2+n3*phat3_p2
    pdotvr_p2 = r*(phat1_p2*v1+phat2_p2*v2+phat3_p2*v3)
    pphi_p2 = pdotxir_p2
    DrSipn2_p2 = 2*Deltar*pdotn*pdotn_p2/Sigma
    Q_p2 = DrSipn2_p2+2*Qcoeff1*pdotxir*pdotxir_p2+2*Qcoeff2*pdotvr*pdotvr_p2
    Qminus1_p2 = Q_p2
    sigmacoeffTerm2_p2 = eta*(eta*r*(-882*DrSipn2_p2+204*Qminus1_p2+r*(1620*DrSipn2*DrSipn2_p2-234*DrSipn2*Qminus1_p2-234*DrSipn2_p2*Qminus1))+r*(-96*DrSipn2_p2-436*Qminus1_p2+r*(36*DrSipn2*Qminus1_p2+36*DrSipn2_p2*Qminus1-90*Qminus1*Qminus1_p2)))/(144*r**2)
    sigmacoeffTerm1_p2 = eta*(-36*DrSipn2_p2+3*Qminus1_p2)/12
    sigmacoeff_p2 = sigmacoeffTerm1_p2+sigmacoeffTerm2_p2
    sigmastarcoeffTerm2_p2 = eta*(eta*r*(-324*DrSipn2_p2+120*Qminus1_p2+r*(720*DrSipn2*DrSipn2_p2+Qminus1*(-126*DrSipn2_p2-3*Qminus1_p2)+Qminus1_p2*(-126*DrSipn2-3*Qminus1)))+r*(282*DrSipn2_p2+Qminus1*r*(96*DrSipn2_p2-23*Qminus1_p2)+Qminus1_p2*r*(96*DrSipn2-23*Qminus1)-206*Qminus1_p2))/(72*r**2)
    sigmastarcoeffTerm1_p2 = eta*(-30*DrSipn2_p2+4*Qminus1_p2)/12
    sigmastarcoeff_p2 = sigmastarcoeffTerm1_p2+sigmastarcoeffTerm2_p2
    Deltasigmastar3_p2 = sigma3*sigmacoeff_p2+sigmastar3*sigmastarcoeff_p2
    Deltasigmastar2_p2 = sigma2*sigmacoeff_p2+sigmastar2*sigmastarcoeff_p2
    Deltasigmastar1_p2 = sigma1*sigmacoeff_p2+sigmastar1*sigmastarcoeff_p2
    Sstar3_p2 = Deltasigmastar3_p2
    Sstar2_p2 = Deltasigmastar2_p2
    Sstar1_p2 = Deltasigmastar1_p2
    S3_p2 = Sstar3_p2
    S2_p2 = Sstar2_p2
    S1_p2 = Sstar1_p2
    Sstardotn_p2 = Sstar1_p2*n1+Sstar2_p2*n2+Sstar3_p2*n3
    SdotSkerrhat_p2 = S1_p2*Skerrhat1+S2_p2*Skerrhat2+S3_p2*Skerrhat3
    Sdotn_p2 = S1_p2*n1+S2_p2*n2+S3_p2*n3
    Sdotv_p2 = S1_p2*v1+S2_p2*v2+S3_p2*v3
    Sdotxi_p2 = S1_p2*xi1+S2_p2*xi2+S3_p2*xi3
    HdsumTerm2_p2 = 6*Sstardotn*Sstardotn_p2
    HdsumTerm1_p2 = 2*Sstar1*Sstar1_p2+2*Sstar2*Sstar2_p2+2*Sstar3*Sstar3_p2
    Hdsum_p2 = HdsumTerm1_p2-HdsumTerm2_p2
    Q4_p2 = 8*eta*prT**3*prT_p2*u**2*(-3*eta+4)
    gammappsum_p2 = 2*Deltar*pdotn*pdotn_p2/Sigma+2*pdotvr*pdotvr_p2/(Sigma*sin2theta)+2*Sigma*pdotxir*pdotxir_p2/(Lambt*sin2theta)
    Hnsradicand_p2 = Q4_p2+gammappsum_p2
    betapsum_p2 = omegatilde*pphi_p2/Lambt
    HssTerm3_p2 = Btilde**2*(Sdotn*exp2mu*xisq*(-Q_p2-Q_p2/(2*np.sqrt(Q)))+Sdotn_p2*exp2mu*xisq*(-np.sqrt(Q)-Q)+pdotvr*(-Jtilde*Sdotv*pdotn_p2-Jtilde*Sdotv_p2*pdotn+Sdotn*pdotvr_p2+Sdotn_p2*pdotvr)+pdotvr_p2*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr))+expmu*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn_p2+Btilde*Jtilde*Sdotxi_p2*pdotn-Sdotn*expmu*expnu*pdotxir_p2-Sdotn_p2*expmu*expnu*pdotxir)+expmu*expnu*pdotxir_p2*(Btilde*Jtilde*Sdotxi*pdotn-Sdotn*expmu*expnu*pdotxir)
    HssTerm3coeff_p2 = omegacostheta*(-Q_p2-Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q)+Q)**2)
    HssTerm2_p2 = Btilde**2*xisq*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotn_p2-Jtilde*Sdotv_p2*pdotn+Sdotn*pdotvr_p2+Sdotn_p2*pdotvr)+Jtilde*pdotn_p2*(-Jtilde*Sdotv*pdotn+Sdotn*pdotvr)+Sdotv*exp2mu*(Q_p2+Q_p2/(2*np.sqrt(Q)))+Sdotv_p2*exp2mu*(np.sqrt(Q)+Q))+expmu*pdotxir*(-Btilde*Sdotxi*expnu*pdotvr_p2-Btilde*Sdotxi_p2*expnu*pdotvr+Sdotv*exp2nu*expmu*pdotxir_p2+Sdotv_p2*exp2nu*expmu*pdotxir)+expmu*pdotxir_p2*(-Btilde*Sdotxi*expnu*pdotvr+Sdotv*exp2nu*expmu*pdotxir)
    HssTerm2coeff_p2 = Jtilde*omegar*(-Q_p2-Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q)+Q)**2)
    HssTerm1_p2 = SdotSkerrhat_p2*omega
    Hss_p2 = HssTerm1_p2+HssTerm2*HssTerm2coeff_p2+HssTerm2coeff*HssTerm2_p2+HssTerm3*HssTerm3coeff_p2+HssTerm3coeff*HssTerm3_p2
    HsoTerm2c_p2 = Brtilde*Jtilde*Sdotv*expmu*expnu*pdotxir_p2*(np.sqrt(Q)+1)+Brtilde*Jtilde*Sdotv_p2*expmu*expnu*pdotxir*(np.sqrt(Q)+1)+Brtilde*Jtilde*Q_p2*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q))
    HsoTerm2b_p2 = Btilde*expmu*expnu*pdotxir*(2*np.sqrt(Q)+1)*(Jtilde*Sdotv_p2*nur-Sdotn_p2*nucostheta*xisq)+Btilde*expmu*expnu*pdotxir_p2*(2*np.sqrt(Q)+1)*(Jtilde*Sdotv*nur-Sdotn*nucostheta*xisq)+Btilde*Q_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur-Sdotn*nucostheta*xisq)/np.sqrt(Q)
    HsoTerm2a_p2 = Btilde**2*Jtilde*Sdotxi*(-np.sqrt(Q)*(nur*pdotvr_p2+pdotn_p2*xisq*(mucostheta-nucostheta))-mucostheta*pdotn_p2*xisq+mur*pdotvr_p2*(np.sqrt(Q)+1)+Q_p2*mur*pdotvr/(2*np.sqrt(Q))+Q_p2*(-nur*pdotvr-pdotn*xisq*(mucostheta-nucostheta))/(2*np.sqrt(Q)))+Btilde**2*Jtilde*Sdotxi_p2*(np.sqrt(Q)*(-nur*pdotvr-pdotn*xisq*(mucostheta-nucostheta))-mucostheta*pdotn*xisq+mur*pdotvr*(np.sqrt(Q)+1))
    HsoTerm2_p2 = HsoTerm2a_p2+HsoTerm2b_p2-HsoTerm2c_p2
    HsoTerm2coeff_p2 = expnu*(-Q_p2-Q_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q)+Q)**2)
    HsoTerm1_p2 = SdotSkerrhat*exp2nu*pdotxir_p2*(-Btilde+expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq)+SdotSkerrhat_p2*exp2nu*pdotxir*(-Btilde+expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq)-Q_p2*SdotSkerrhat*exp2nu*pdotxir*(-Btilde+expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq)
    Hso_p2 = HsoTerm1_p2+HsoTerm2*HsoTerm2coeff_p2+HsoTerm2coeff*HsoTerm2_p2
    Hd_p2 = Hdcoeff*Hdsum_p2
    Hns_p2 = betapsum_p2+Hnsradicand_p2*alpha/(2*np.sqrt(Hnsradicand))
    Hs_p2 = Hso_p2+Hss_p2
    Heff_p2 = -Hd_p2+Hns_p2+Hs_p2
    rprm_x = x/np.sqrt(x**2 + y**2 + z**2)
    uprm_x = -rprm_x/r**2
    n3prm_x = -rprm_x*z/r**2
    n2prm_x = -rprm_x*y/r**2
    n1prm_x = 1/r - rprm_x*x/r**2
    xi3prm_x = e31*n2prm_x - e32*n1prm_x
    xi2prm_x = -e31*n3prm_x + e33*n1prm_x
    xi1prm_x = e32*n3prm_x - e33*n2prm_x
    v3prm_x = n1*xi2prm_x + n1prm_x*xi2 - n2*xi1prm_x - n2prm_x*xi1
    v2prm_x = -n1*xi3prm_x - n1prm_x*xi3 + n3*xi1prm_x + n3prm_x*xi1
    v1prm_x = n2*xi3prm_x + n2prm_x*xi3 - n3*xi2prm_x - n3prm_x*xi2
    costhetaprm_x = e31*n1prm_x + e32*n2prm_x + e33*n3prm_x
    sin2thetaprm_x = -2*costheta*costhetaprm_x
    xisqprm_x = sin2thetaprm_x
    w2prm_x = 2*r*rprm_x
    Sigmaprm_x = 2*a**2*costheta*costhetaprm_x + 2*r*rprm_x
    Dinvprm_x = (3*eta*u**2*uprm_x*(-6*eta + 52) + 12*eta*u*uprm_x)/(eta*u**3*(-6*eta + 52) + 6*eta*u**2 + 1)
    omegatildeprm_x = 2*a*rprm_x
    logargprm_x = u*(u*(u*(u*(Delta5l*uprm_x + uprm_x*(Delta5 + Delta5l*np.log(u))) + uprm_x*(Delta4 + u*(Delta5 + Delta5l*np.log(u)))) + uprm_x*(Delta3 + u*(Delta4 + u*(Delta5 + Delta5l*np.log(u))))) + uprm_x*(Delta2 + u*(Delta3 + u*(Delta4 + u*(Delta5 + Delta5l*np.log(u)))))) + uprm_x*(Delta1 + u*(Delta2 + u*(Delta3 + u*(Delta4 + u*(Delta5 + Delta5l*np.log(u))))))
    Deltaucalibprm_x = eta*logargprm_x/(logarg + 1)
    Deltaucalibprm_x = eta*logargprm_x*u**2*(Delta1 + u*(2*Delta2 + u*(3*Delta3 + u*(4*Delta4 + u*(5*Delta5 + 5*Delta5l*np.log(u))))))/(logarg + 1)**2 - eta*u**2*(u*(u*(u*(5*Delta5l*uprm_x + uprm_x*(5*Delta5 + 5*Delta5l*np.log(u))) + uprm_x*(4*Delta4 + u*(5*Delta5 + 5*Delta5l*np.log(u)))) + uprm_x*(3*Delta3 + u*(4*Delta4 + u*(5*Delta5 + 5*Delta5l*np.log(u))))) + uprm_x*(2*Delta2 + u*(3*Delta3 + u*(4*Delta4 + u*(5*Delta5 + 5*Delta5l*np.log(u))))))/(logarg + 1) - 2*eta*u*uprm_x*(Delta1 + u*(2*Delta2 + u*(3*Delta3 + u*(4*Delta4 + u*(5*Delta5 + 5*Delta5l*np.log(u))))))/(logarg + 1)
    Deltaubarprm_x = 2*a**2*u*uprm_x + 2*uprm_x/etaKminus1
    Deltaubarprm_x = -6*a**2*u**2*uprm_x - 4*u*uprm_x/etaKminus1
    Deltauprm_x = Deltaubar*Deltaucalibprm_x + Deltaubarprm_x*Deltaucalib
    Deltauprm_x = 2*Deltaubar*Deltaucalibprm_x + 2*Deltaubarprm_x*Deltaucalib
    Deltatprm_x = 2*Deltau*r*rprm_x + 2*Deltau*rprm_x + Deltauprm_x*r**2 + 2*Deltauprm_x*r
    return "2*Deltau*r*rprm_x + 2*Deltau*rprm_x + Deltauprm_x*r**2 + 2*Deltauprm_x*r"
    Deltatprm_x = 2*Deltau*r*rprm_x + Deltauprm_x*r**2
    Deltarprm_x = Deltat*Dinvprm_x + Deltatprm_x*Dinv
    Lambtprm_x = -Deltat*a**2*sin2thetaprm_x - Deltatprm_x*a**2*sin2theta + 2*w2*w2prm_x
    csiprm_x = -w2prm_x*np.sqrt(Deltar*Deltat)/w2**2 + np.sqrt(Deltar*Deltat)*(Deltar*Deltatprm_x/2 + Deltarprm_x*Deltat/2)/(Deltar*Deltat*w2)
    csi1prm_x = csiprm_x*(-abs(-tortoise + 1) + 1)
    csi2prm_x = csiprm_x*(-np.sign(-tortoise + 3/2)/2 + 1/2)
    prTprm_x = csi2*(n1prm_x*p1 + n2prm_x*p2 + n3prm_x*p3) + csi2prm_x*(n1*p1 + n2*p2 + n3*p3)
    phat3prm_x = n3*prTprm_x*(1 - 1/csi1) + n3prm_x*prT*(1 - 1/csi1) + csi1prm_x*n3*prT/csi1**2
    phat2prm_x = n2*prTprm_x*(1 - 1/csi1) + n2prm_x*prT*(1 - 1/csi1) + csi1prm_x*n2*prT/csi1**2
    phat1prm_x = n1*prTprm_x*(1 - 1/csi1) + n1prm_x*prT*(1 - 1/csi1) + csi1prm_x*n1*prT/csi1**2
    pdotxirprm_x = r*(phat1*xi1prm_x + phat1prm_x*xi1 + phat2*xi2prm_x + phat2prm_x*xi2 + phat3*xi3prm_x + phat3prm_x*xi3) + rprm_x*(phat1*xi1 + phat2*xi2 + phat3*xi3)
    pdotnprm_x = n1*phat1prm_x + n1prm_x*phat1 + n2*phat2prm_x + n2prm_x*phat2 + n3*phat3prm_x + n3prm_x*phat3
    pdotvrprm_x = r*(phat1*v1prm_x + phat1prm_x*v1 + phat2*v2prm_x + phat2prm_x*v2 + phat3*v3prm_x + phat3prm_x*v3) + rprm_x*(phat1*v1 + phat2*v2 + phat3*v3)
    pphiprm_x = pdotxirprm_x
    Qcoeff2prm_x = -sin2thetaprm_x/(Sigma*sin2theta**2) - Sigmaprm_x/(Sigma**2*sin2theta)
    Qcoeff1prm_x = -Sigma*sin2thetaprm_x/(Lambt*sin2theta**2) + Sigmaprm_x/(Lambt*sin2theta) - Lambtprm_x*Sigma/(Lambt**2*sin2theta)
    DrSipn2prm_x = 2*Deltar*pdotn*pdotnprm_x/Sigma - Deltar*Sigmaprm_x*pdotn**2/Sigma**2 + Deltarprm_x*pdotn**2/Sigma
    Qprm_x = DrSipn2prm_x + 2*Qcoeff1*pdotxir*pdotxirprm_x + Qcoeff1prm_x*pdotxir**2 + 2*Qcoeff2*pdotvr*pdotvrprm_x + Qcoeff2prm_x*pdotvr**2
    Qminus1prm_x = Qprm_x
    Jtildeprm_x = Deltarprm_x/(2*np.sqrt(Deltar))
    exp2muprm_x = Sigmaprm_x
    expmuprm_x = exp2muprm_x/(2*np.sqrt(exp2mu))
    Brtildeprm_x = (np.sqrt(Deltar)*Deltatprm_x - 2*Deltatprm_x + Deltarprm_x*Deltat/(2*np.sqrt(Deltar)))/(2*np.sqrt(Deltar*Deltat)) + (np.sqrt(Deltar)*Deltat - 2*Deltat)*(-Deltar*Deltatprm_x/2 - Deltarprm_x*Deltat/2)/(2*Deltar*Deltat*np.sqrt(Deltar*Deltat))
    Btildeprm_x = Deltatprm_x/(2*np.sqrt(Deltat))
    exp2nuprm_x = Deltat*Sigmaprm_x/Lambt - Deltat*Lambtprm_x*Sigma/Lambt**2 + Deltatprm_x*Sigma/Lambt
    expnuprm_x = exp2nuprm_x/(2*np.sqrt(exp2nu))
    omegaprm_x = omegatildeprm_x/Lambt - Lambtprm_x*omegatilde/Lambt**2
    Lambtprm_x = -2*Deltat*a**2*sin2thetaprm_x - 2*Deltatprm_x*a**2*sin2theta + 8*r**2*rprm_x + rprm_x*(4*a**2 + 4*r**2)
    mucosthetaprm_x = a**2*costhetaprm_x/Sigma - Sigmaprm_x*a**2*costheta/Sigma**2
    nucosthetaprm_x = a**2*costheta*w2*(-Deltatprm_x + w2prm_x)/(Lambt*Sigma) + a**2*costheta*w2prm_x*(-Deltat + w2)/(Lambt*Sigma) + a**2*costhetaprm_x*w2*(-Deltat + w2)/(Lambt*Sigma) - Sigmaprm_x*a**2*costheta*w2*(-Deltat + w2)/(Lambt*Sigma**2) - Lambtprm_x*a**2*costheta*w2*(-Deltat + w2)/(Lambt**2*Sigma)
    omegacosthetaprm_x = -2*Deltat*a**2*costhetaprm_x*omegatilde/Lambt**2 + 4*Deltat*Lambtprm_x*a**2*costheta*omegatilde/Lambt**3 - 2*Deltatprm_x*a**2*costheta*omegatilde/Lambt**2
    murprm_x = rprm_x/Sigma - Sigmaprm_x*r/Sigma**2 + Deltarprm_x/(2*Deltar**(3/2))
    nurprm_x = rprm_x/Sigma - Sigmaprm_x*r/Sigma**2 + w2*(-4*Deltat*rprm_x + Deltat*w2prm_x - 4*Deltatprm_x*r + Deltatprm_x*w2)/(2*Deltat*Lambt) + w2prm_x*(-4*Deltat*r + Deltat*w2)/(2*Deltat*Lambt) - Lambtprm_x*w2*(-4*Deltat*r + Deltat*w2)/(2*Deltat*Lambt**2) - Deltatprm_x*w2*(-4*Deltat*r + Deltat*w2)/(2*Deltat**2*Lambt)
    sigmacoeffTerm3prm_x = 3*dSO*eta*u**2*uprm_x
    sigmacoeffTerm2prm_x = eta*(eta*(r*(-882*DrSipn2prm_x + 204*Qminus1prm_x + r*(1620*DrSipn2*DrSipn2prm_x - 234*DrSipn2*Qminus1prm_x - 234*DrSipn2prm_x*Qminus1) + rprm_x*(810*DrSipn2**2 - 234*DrSipn2*Qminus1)) + rprm_x*(-882*DrSipn2 + 204*Qminus1 + r*(810*DrSipn2**2 - 234*DrSipn2*Qminus1))) + r*(-96*DrSipn2prm_x - 436*Qminus1prm_x + r*(36*DrSipn2*Qminus1prm_x + 36*DrSipn2prm_x*Qminus1 - 90*Qminus1*Qminus1prm_x) + rprm_x*(36*DrSipn2*Qminus1 - 45*Qminus1**2)) + rprm_x*(-96*DrSipn2 - 436*Qminus1 + r*(36*DrSipn2*Qminus1 - 45*Qminus1**2)))/(144*r**2) - eta*rprm_x*(eta*(r*(-882*DrSipn2 + 204*Qminus1 + r*(810*DrSipn2**2 - 234*DrSipn2*Qminus1)) - 336) + r*(-96*DrSipn2 - 436*Qminus1 + r*(36*DrSipn2*Qminus1 - 45*Qminus1**2)) - 896)/(72*r**3)
    sigmacoeffTerm1prm_x = eta*(-36*DrSipn2prm_x + 3*Qminus1prm_x + 8*rprm_x/r**2)/12
    sigmacoeffprm_x = sigmacoeffTerm1prm_x + sigmacoeffTerm2prm_x + sigmacoeffTerm3prm_x
    sigmastarcoeffTerm2prm_x = eta*(eta*(r*(-324*DrSipn2prm_x + 120*Qminus1prm_x + r*(720*DrSipn2*DrSipn2prm_x + Qminus1*(-126*DrSipn2prm_x - 3*Qminus1prm_x) + Qminus1prm_x*(-126*DrSipn2 - 3*Qminus1)) + rprm_x*(360*DrSipn2**2 + Qminus1*(-126*DrSipn2 - 3*Qminus1))) + rprm_x*(-324*DrSipn2 + 120*Qminus1 + r*(360*DrSipn2**2 + Qminus1*(-126*DrSipn2 - 3*Qminus1)))) + r*(282*DrSipn2prm_x + Qminus1*r*(96*DrSipn2prm_x - 23*Qminus1prm_x) + Qminus1*rprm_x*(96*DrSipn2 - 23*Qminus1) + Qminus1prm_x*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1prm_x) + rprm_x*(282*DrSipn2 + Qminus1*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1))/(72*r**2) - eta*rprm_x*(eta*(r*(-324*DrSipn2 + 120*Qminus1 + r*(360*DrSipn2**2 + Qminus1*(-126*DrSipn2 - 3*Qminus1))) - 54) + r*(282*DrSipn2 + Qminus1*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1) + 706)/(36*r**3)
    sigmastarcoeffTerm1prm_x = eta*(-30*DrSipn2prm_x + 4*Qminus1prm_x - 14*rprm_x/r**2)/12
    sigmastarcoeffprm_x = sigmastarcoeffTerm1prm_x + sigmastarcoeffTerm2prm_x
    Deltasigmastar3prm_x = sigma3*sigmacoeffprm_x + sigmastar3*sigmastarcoeffprm_x
    Deltasigmastar2prm_x = sigma2*sigmacoeffprm_x + sigmastar2*sigmastarcoeffprm_x
    Deltasigmastar1prm_x = sigma1*sigmacoeffprm_x + sigmastar1*sigmastarcoeffprm_x
    Sstar3prm_x = Deltasigmastar3prm_x
    Sstar2prm_x = Deltasigmastar2prm_x
    Sstar1prm_x = Deltasigmastar1prm_x
    S3prm_x = Sstar3prm_x
    S2prm_x = Sstar2prm_x
    S1prm_x = Sstar1prm_x
    Sstardotnprm_x = Sstar1*n1prm_x + Sstar1prm_x*n1 + Sstar2*n2prm_x + Sstar2prm_x*n2 + Sstar3*n3prm_x + Sstar3prm_x*n3
    SdotSkerrhatprm_x = S1prm_x*Skerrhat1 + S2prm_x*Skerrhat2 + S3prm_x*Skerrhat3
    Sdotnprm_x = S1*n1prm_x + S1prm_x*n1 + S2*n2prm_x + S2prm_x*n2 + S3*n3prm_x + S3prm_x*n3
    Sdotvprm_x = S1*v1prm_x + S1prm_x*v1 + S2*v2prm_x + S2prm_x*v2 + S3*v3prm_x + S3prm_x*v3
    Sdotxiprm_x = S1*xi1prm_x + S1prm_x*xi1 + S2*xi2prm_x + S2prm_x*xi2 + S3*xi3prm_x + S3prm_x*xi3
    HdsumTerm2prm_x = 6*Sstardotn*Sstardotnprm_x
    HdsumTerm1prm_x = 2*Sstar1*Sstar1prm_x + 2*Sstar2*Sstar2prm_x + 2*Sstar3*Sstar3prm_x
    Hdsumprm_x = HdsumTerm1prm_x - HdsumTerm2prm_x
    Hdcoeffprm_x = -3*rprm_x/(2*r**4)
    Q4prm_x = 4*eta*prT**4*u*uprm_x*(-3*eta + 4) + 8*eta*prT**3*prTprm_x*u**2*(-3*eta + 4)
    gammappsumprm_x = 2*Deltar*pdotn*pdotnprm_x/Sigma - Deltar*Sigmaprm_x*pdotn**2/Sigma**2 + Deltarprm_x*pdotn**2/Sigma - pdotvr**2*sin2thetaprm_x/(Sigma*sin2theta**2) + 2*pdotvr*pdotvrprm_x/(Sigma*sin2theta) - Sigmaprm_x*pdotvr**2/(Sigma**2*sin2theta) - Sigma*pdotxir**2*sin2thetaprm_x/(Lambt*sin2theta**2) + 2*Sigma*pdotxir*pdotxirprm_x/(Lambt*sin2theta) + Sigmaprm_x*pdotxir**2/(Lambt*sin2theta) - Lambtprm_x*Sigma*pdotxir**2/(Lambt**2*sin2theta)
    Hnsradicandprm_x = Q4prm_x + gammappsumprm_x
    alphaprm_x = Lambt*np.sqrt(Deltat*Sigma/Lambt)*(Deltat*Sigmaprm_x/(2*Lambt) - Deltat*Lambtprm_x*Sigma/(2*Lambt**2) + Deltatprm_x*Sigma/(2*Lambt))/(Deltat*Sigma)
    betapsumprm_x = omegatilde*pphiprm_x/Lambt - Lambtprm_x*omegatilde*pphi/Lambt**2
    HssTerm3prm_x = Btilde**2*(Sdotn*exp2mu*xisq*(-Qprm_x - Qprm_x/(2*np.sqrt(Q))) + Sdotn*exp2mu*xisqprm_x*(-np.sqrt(Q) - Q) + Sdotn*exp2muprm_x*xisq*(-np.sqrt(Q) - Q) + Sdotnprm_x*exp2mu*xisq*(-np.sqrt(Q) - Q) + pdotvr*(-Jtilde*Sdotv*pdotnprm_x - Jtilde*Sdotvprm_x*pdotn - Jtildeprm_x*Sdotv*pdotn + Sdotn*pdotvrprm_x + Sdotnprm_x*pdotvr) + pdotvrprm_x*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr)) + Btilde*Btildeprm_x*(-2*Sdotn*exp2mu*xisq*(np.sqrt(Q) + Q) + 2*pdotvr*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr)) + expmu*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotnprm_x + Btilde*Jtilde*Sdotxiprm_x*pdotn + Btilde*Jtildeprm_x*Sdotxi*pdotn + Btildeprm_x*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxirprm_x - Sdotn*expmu*expnuprm_x*pdotxir - Sdotn*expmuprm_x*expnu*pdotxir - Sdotnprm_x*expmu*expnu*pdotxir) + expmu*expnu*pdotxirprm_x*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir) + expmu*expnuprm_x*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir) + expmuprm_x*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir)
    HssTerm3coeffprm_x = omegacostheta*(-Qprm_x - Qprm_x/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**2) + omegacosthetaprm_x/(Btilde*exp2mu*expmu*expnu*(2*np.sqrt(Q) + 2*Q)) - expnuprm_x*omegacostheta/(Btilde*exp2mu*expmu*expnu**2*(2*np.sqrt(Q) + 2*Q)) - expmuprm_x*omegacostheta/(Btilde*exp2mu*expmu**2*expnu*(2*np.sqrt(Q) + 2*Q)) - exp2muprm_x*omegacostheta/(Btilde*exp2mu**2*expmu*expnu*(2*np.sqrt(Q) + 2*Q)) - Btildeprm_x*omegacostheta/(Btilde**2*exp2mu*expmu*expnu*(2*np.sqrt(Q) + 2*Q))
    HssTerm2prm_x = Btilde**2*xisq*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotnprm_x - Jtilde*Sdotvprm_x*pdotn - Jtildeprm_x*Sdotv*pdotn + Sdotn*pdotvrprm_x + Sdotnprm_x*pdotvr) + Jtilde*pdotnprm_x*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Jtildeprm_x*pdotn*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Sdotv*exp2mu*(Qprm_x + Qprm_x/(2*np.sqrt(Q))) + Sdotv*exp2muprm_x*(np.sqrt(Q) + Q) + Sdotvprm_x*exp2mu*(np.sqrt(Q) + Q)) + Btilde**2*xisqprm_x*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Sdotv*exp2mu*(np.sqrt(Q) + Q)) + Btilde*Btildeprm_x*xisq*(2*Jtilde*pdotn*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + 2*Sdotv*exp2mu*(np.sqrt(Q) + Q)) + expmu*pdotxir*(-Btilde*Sdotxi*expnu*pdotvrprm_x - Btilde*Sdotxi*expnuprm_x*pdotvr - Btilde*Sdotxiprm_x*expnu*pdotvr - Btildeprm_x*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxirprm_x + Sdotv*exp2nu*expmuprm_x*pdotxir + Sdotv*exp2nuprm_x*expmu*pdotxir + Sdotvprm_x*exp2nu*expmu*pdotxir) + expmu*pdotxirprm_x*(-Btilde*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir) + expmuprm_x*pdotxir*(-Btilde*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir)
    HssTerm2coeffprm_x = Jtilde*omegar*(-Qprm_x - Qprm_x/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**2) - Jtilde*omegar*xisqprm_x/(Btilde*exp2mu*expmu*expnu*xisq**2*(2*np.sqrt(Q) + 2*Q)) - Jtilde*expnuprm_x*omegar/(Btilde*exp2mu*expmu*expnu**2*xisq*(2*np.sqrt(Q) + 2*Q)) - Jtilde*expmuprm_x*omegar/(Btilde*exp2mu*expmu**2*expnu*xisq*(2*np.sqrt(Q) + 2*Q)) - Jtilde*exp2muprm_x*omegar/(Btilde*exp2mu**2*expmu*expnu*xisq*(2*np.sqrt(Q) + 2*Q)) + Jtildeprm_x*omegar/(Btilde*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q) + 2*Q)) - Btildeprm_x*Jtilde*omegar/(Btilde**2*exp2mu*expmu*expnu*xisq*(2*np.sqrt(Q) + 2*Q))
    HssTerm1prm_x = SdotSkerrhat*omegaprm_x + SdotSkerrhatprm_x*omega
    Hssprm_x = HssTerm1prm_x + HssTerm2*HssTerm2coeffprm_x + HssTerm2coeff*HssTerm2prm_x + HssTerm3*HssTerm3coeffprm_x + HssTerm3coeff*HssTerm3prm_x
    HsoTerm2cprm_x = Brtilde*Jtilde*Sdotv*expmu*expnu*pdotxirprm_x*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv*expmu*expnuprm_x*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv*expmuprm_x*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotvprm_x*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Qprm_x*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtildeprm_x*Sdotv*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtildeprm_x*Jtilde*Sdotv*expmu*expnu*pdotxir*(np.sqrt(Q) + 1)
    HsoTerm2bprm_x = Btilde*expmu*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nurprm_x + Jtilde*Sdotvprm_x*nur + Jtildeprm_x*Sdotv*nur - Sdotn*nucostheta*xisqprm_x - Sdotn*nucosthetaprm_x*xisq - Sdotnprm_x*nucostheta*xisq) + Btilde*expmu*expnu*pdotxirprm_x*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*expmu*expnuprm_x*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*expmuprm_x*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*Qprm_x*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btildeprm_x*expmu*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)
    HsoTerm2aprm_x = Btilde**2*Jtilde*Sdotxi*(-np.sqrt(Q)*(nur*pdotvrprm_x + nurprm_x*pdotvr + pdotn*xisq*(mucosthetaprm_x - nucosthetaprm_x) + pdotn*xisqprm_x*(mucostheta - nucostheta) + pdotnprm_x*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn*xisqprm_x - mucostheta*pdotnprm_x*xisq - mucosthetaprm_x*pdotn*xisq + mur*pdotvrprm_x*(np.sqrt(Q) + 1) + murprm_x*pdotvr*(np.sqrt(Q) + 1) + Qprm_x*mur*pdotvr/(2*np.sqrt(Q)) + Qprm_x*(-nur*pdotvr - pdotn*xisq*(mucostheta - nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtilde*Sdotxiprm_x*(np.sqrt(Q)*(-nur*pdotvr - pdotn*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn*xisq + mur*pdotvr*(np.sqrt(Q) + 1)) + Btilde**2*Jtildeprm_x*Sdotxi*(np.sqrt(Q)*(-nur*pdotvr - pdotn*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn*xisq + mur*pdotvr*(np.sqrt(Q) + 1)) + Btilde*Btildeprm_x*Jtilde*Sdotxi*(2*np.sqrt(Q)*(-nur*pdotvr - pdotn*xisq*(mucostheta - nucostheta)) - 2*mucostheta*pdotn*xisq + 2*mur*pdotvr*(np.sqrt(Q) + 1))
    HsoTerm2prm_x = HsoTerm2aprm_x + HsoTerm2bprm_x - HsoTerm2cprm_x
    return HsoTerm2aprm_x
    HsoTerm2coeffprm_x = expnu*(-Qprm_x - Qprm_x/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**2) - expnu*xisqprm_x/(Btilde**2*exp2mu*xisq**2*(np.sqrt(Q) + Q)) + expnuprm_x/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)) - exp2muprm_x*expnu/(Btilde**2*exp2mu**2*xisq*(np.sqrt(Q) + Q)) - 2*Btildeprm_x*expnu/(Btilde**3*exp2mu*xisq*(np.sqrt(Q) + Q))
    HsoTerm1prm_x = SdotSkerrhat*exp2nu*pdotxir*(-Btildeprm_x + expmu*expnuprm_x + expmuprm_x*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat*exp2nu*pdotxir*xisqprm_x*(Btilde - expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq**2) + SdotSkerrhat*exp2nu*pdotxirprm_x*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - SdotSkerrhat*exp2nu*expmuprm_x*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu**2*xisq) + SdotSkerrhat*exp2nuprm_x*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhatprm_x*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - Qprm_x*SdotSkerrhat*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Btildeprm_x*SdotSkerrhat*exp2nu*pdotxir*(-2*Btilde + 2*expmu*expnu)/(Btilde**3*np.sqrt(Q)*expmu*xisq)
    Hsoprm_x = HsoTerm1prm_x + HsoTerm2*HsoTerm2coeffprm_x + HsoTerm2coeff*HsoTerm2prm_x
    Hdprm_x = Hdcoeff*Hdsumprm_x + Hdcoeffprm_x*Hdsum
    Hnsprm_x = np.sqrt(Hnsradicand)*alphaprm_x + betapsumprm_x + Hnsradicandprm_x*alpha/(2*np.sqrt(Hnsradicand))
    Hsprm_x = Hsoprm_x + Hssprm_x
    Heffprm_x = -Hdprm_x + Hnsprm_x + Hsprm_x + 4*dSS*eta*u**3*uprm_x*(S1x**2 + S1y**2 + S1z**2 + S2x**2 + S2y**2 + S2z**2)
    prT_p2prm_x = csi2*n2prm_x + csi2prm_x*n2
    phat3_p2prm_x = n3*prT_p2prm_x*(1 - 1/csi1) + n3prm_x*prT_p2*(1 - 1/csi1) + csi1prm_x*n3*prT_p2/csi1**2
    phat2_p2prm_x = n2*prT_p2prm_x*(1 - 1/csi1) + n2prm_x*prT_p2*(1 - 1/csi1) + csi1prm_x*n2*prT_p2/csi1**2
    phat1_p2prm_x = n1*prT_p2prm_x*(1 - 1/csi1) + n1prm_x*prT_p2*(1 - 1/csi1) + csi1prm_x*n1*prT_p2/csi1**2
    pdotxir_p2prm_x = r*(phat1_p2*xi1prm_x + phat1_p2prm_x*xi1 + phat2_p2*xi2prm_x + phat2_p2prm_x*xi2 + phat3_p2*xi3prm_x + phat3_p2prm_x*xi3) + rprm_x*(phat1_p2*xi1 + phat2_p2*xi2 + phat3_p2*xi3)
    pdotn_p2prm_x = n1*phat1_p2prm_x + n1prm_x*phat1_p2 + n2*phat2_p2prm_x + n2prm_x*phat2_p2 + n3*phat3_p2prm_x + n3prm_x*phat3_p2
    pdotvr_p2prm_x = r*(phat1_p2*v1prm_x + phat1_p2prm_x*v1 + phat2_p2*v2prm_x + phat2_p2prm_x*v2 + phat3_p2*v3prm_x + phat3_p2prm_x*v3) + rprm_x*(phat1_p2*v1 + phat2_p2*v2 + phat3_p2*v3)
    pphi_p2prm_x = pdotxir_p2prm_x
    DrSipn2_p2prm_x = 2*Deltar*pdotn*pdotn_p2prm_x/Sigma + 2*Deltar*pdotn_p2*pdotnprm_x/Sigma - 2*Deltar*Sigmaprm_x*pdotn*pdotn_p2/Sigma**2 + 2*Deltarprm_x*pdotn*pdotn_p2/Sigma
    Q_p2prm_x = DrSipn2_p2prm_x + 2*Qcoeff1*pdotxir*pdotxir_p2prm_x + 2*Qcoeff1*pdotxir_p2*pdotxirprm_x + 2*Qcoeff1prm_x*pdotxir*pdotxir_p2 + 2*Qcoeff2*pdotvr*pdotvr_p2prm_x + 2*Qcoeff2*pdotvr_p2*pdotvrprm_x + 2*Qcoeff2prm_x*pdotvr*pdotvr_p2
    Qminus1_p2prm_x = Q_p2prm_x
    sigmacoeffTerm2_p2prm_x = eta*(eta*r*(-882*DrSipn2_p2prm_x + 204*Qminus1_p2prm_x + r*(1620*DrSipn2*DrSipn2_p2prm_x - 234*DrSipn2*Qminus1_p2prm_x + 1620*DrSipn2_p2*DrSipn2prm_x - 234*DrSipn2_p2*Qminus1prm_x - 234*DrSipn2_p2prm_x*Qminus1 - 234*DrSipn2prm_x*Qminus1_p2) + rprm_x*(1620*DrSipn2*DrSipn2_p2 - 234*DrSipn2*Qminus1_p2 - 234*DrSipn2_p2*Qminus1)) + eta*rprm_x*(-882*DrSipn2_p2 + 204*Qminus1_p2 + r*(1620*DrSipn2*DrSipn2_p2 - 234*DrSipn2*Qminus1_p2 - 234*DrSipn2_p2*Qminus1)) + r*(-96*DrSipn2_p2prm_x - 436*Qminus1_p2prm_x + r*(36*DrSipn2*Qminus1_p2prm_x + 36*DrSipn2_p2*Qminus1prm_x + 36*DrSipn2_p2prm_x*Qminus1 + 36*DrSipn2prm_x*Qminus1_p2 - 90*Qminus1*Qminus1_p2prm_x - 90*Qminus1_p2*Qminus1prm_x) + rprm_x*(36*DrSipn2*Qminus1_p2 + 36*DrSipn2_p2*Qminus1 - 90*Qminus1*Qminus1_p2)) + rprm_x*(-96*DrSipn2_p2 - 436*Qminus1_p2 + r*(36*DrSipn2*Qminus1_p2 + 36*DrSipn2_p2*Qminus1 - 90*Qminus1*Qminus1_p2)))/(144*r**2) - eta*rprm_x*(eta*r*(-882*DrSipn2_p2 + 204*Qminus1_p2 + r*(1620*DrSipn2*DrSipn2_p2 - 234*DrSipn2*Qminus1_p2 - 234*DrSipn2_p2*Qminus1)) + r*(-96*DrSipn2_p2 - 436*Qminus1_p2 + r*(36*DrSipn2*Qminus1_p2 + 36*DrSipn2_p2*Qminus1 - 90*Qminus1*Qminus1_p2)))/(72*r**3)
    sigmacoeffTerm1_p2prm_x = eta*(-36*DrSipn2_p2prm_x + 3*Qminus1_p2prm_x)/12
    sigmacoeff_p2prm_x = sigmacoeffTerm1_p2prm_x + sigmacoeffTerm2_p2prm_x
    sigmastarcoeffTerm2_p2prm_x = eta*(eta*r*(-324*DrSipn2_p2prm_x + 120*Qminus1_p2prm_x + r*(720*DrSipn2*DrSipn2_p2prm_x + 720*DrSipn2_p2*DrSipn2prm_x + Qminus1*(-126*DrSipn2_p2prm_x - 3*Qminus1_p2prm_x) + Qminus1_p2*(-126*DrSipn2prm_x - 3*Qminus1prm_x) + Qminus1_p2prm_x*(-126*DrSipn2 - 3*Qminus1) + Qminus1prm_x*(-126*DrSipn2_p2 - 3*Qminus1_p2)) + rprm_x*(720*DrSipn2*DrSipn2_p2 + Qminus1*(-126*DrSipn2_p2 - 3*Qminus1_p2) + Qminus1_p2*(-126*DrSipn2 - 3*Qminus1))) + eta*rprm_x*(-324*DrSipn2_p2 + 120*Qminus1_p2 + r*(720*DrSipn2*DrSipn2_p2 + Qminus1*(-126*DrSipn2_p2 - 3*Qminus1_p2) + Qminus1_p2*(-126*DrSipn2 - 3*Qminus1))) + r*(282*DrSipn2_p2prm_x + Qminus1*r*(96*DrSipn2_p2prm_x - 23*Qminus1_p2prm_x) + Qminus1*rprm_x*(96*DrSipn2_p2 - 23*Qminus1_p2) + Qminus1_p2*r*(96*DrSipn2prm_x - 23*Qminus1prm_x) + Qminus1_p2*rprm_x*(96*DrSipn2 - 23*Qminus1) + Qminus1_p2prm_x*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1_p2prm_x + Qminus1prm_x*r*(96*DrSipn2_p2 - 23*Qminus1_p2)) + rprm_x*(282*DrSipn2_p2 + Qminus1*r*(96*DrSipn2_p2 - 23*Qminus1_p2) + Qminus1_p2*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1_p2))/(72*r**2) - eta*rprm_x*(eta*r*(-324*DrSipn2_p2 + 120*Qminus1_p2 + r*(720*DrSipn2*DrSipn2_p2 + Qminus1*(-126*DrSipn2_p2 - 3*Qminus1_p2) + Qminus1_p2*(-126*DrSipn2 - 3*Qminus1))) + r*(282*DrSipn2_p2 + Qminus1*r*(96*DrSipn2_p2 - 23*Qminus1_p2) + Qminus1_p2*r*(96*DrSipn2 - 23*Qminus1) - 206*Qminus1_p2))/(36*r**3)
    sigmastarcoeffTerm1_p2prm_x = eta*(-30*DrSipn2_p2prm_x + 4*Qminus1_p2prm_x)/12
    sigmastarcoeff_p2prm_x = sigmastarcoeffTerm1_p2prm_x + sigmastarcoeffTerm2_p2prm_x
    Deltasigmastar3_p2prm_x = sigma3*sigmacoeff_p2prm_x + sigmastar3*sigmastarcoeff_p2prm_x
    Deltasigmastar2_p2prm_x = sigma2*sigmacoeff_p2prm_x + sigmastar2*sigmastarcoeff_p2prm_x
    Deltasigmastar1_p2prm_x = sigma1*sigmacoeff_p2prm_x + sigmastar1*sigmastarcoeff_p2prm_x
    Sstar3_p2prm_x = Deltasigmastar3_p2prm_x
    Sstar2_p2prm_x = Deltasigmastar2_p2prm_x
    Sstar1_p2prm_x = Deltasigmastar1_p2prm_x
    S3_p2prm_x = Sstar3_p2prm_x
    S2_p2prm_x = Sstar2_p2prm_x
    S1_p2prm_x = Sstar1_p2prm_x
    Sstardotn_p2prm_x = Sstar1_p2*n1prm_x + Sstar1_p2prm_x*n1 + Sstar2_p2*n2prm_x + Sstar2_p2prm_x*n2 + Sstar3_p2*n3prm_x + Sstar3_p2prm_x*n3
    SdotSkerrhat_p2prm_x = S1_p2prm_x*Skerrhat1 + S2_p2prm_x*Skerrhat2 + S3_p2prm_x*Skerrhat3
    Sdotn_p2prm_x = S1_p2*n1prm_x + S1_p2prm_x*n1 + S2_p2*n2prm_x + S2_p2prm_x*n2 + S3_p2*n3prm_x + S3_p2prm_x*n3
    Sdotv_p2prm_x = S1_p2*v1prm_x + S1_p2prm_x*v1 + S2_p2*v2prm_x + S2_p2prm_x*v2 + S3_p2*v3prm_x + S3_p2prm_x*v3
    Sdotxi_p2prm_x = S1_p2*xi1prm_x + S1_p2prm_x*xi1 + S2_p2*xi2prm_x + S2_p2prm_x*xi2 + S3_p2*xi3prm_x + S3_p2prm_x*xi3
    HdsumTerm2_p2prm_x = 6*Sstardotn*Sstardotn_p2prm_x + 6*Sstardotn_p2*Sstardotnprm_x
    HdsumTerm1_p2prm_x = 2*Sstar1*Sstar1_p2prm_x + 2*Sstar1_p2*Sstar1prm_x + 2*Sstar2*Sstar2_p2prm_x + 2*Sstar2_p2*Sstar2prm_x + 2*Sstar3*Sstar3_p2prm_x + 2*Sstar3_p2*Sstar3prm_x
    Hdsum_p2prm_x = HdsumTerm1_p2prm_x - HdsumTerm2_p2prm_x
    Q4_p2prm_x = 16*eta*prT**3*prT_p2*u*uprm_x*(-3*eta + 4) + 8*eta*prT**3*prT_p2prm_x*u**2*(-3*eta + 4) + 24*eta*prT**2*prT_p2*prTprm_x*u**2*(-3*eta + 4)
    gammappsum_p2prm_x = 2*Deltar*pdotn*pdotn_p2prm_x/Sigma + 2*Deltar*pdotn_p2*pdotnprm_x/Sigma - 2*Deltar*Sigmaprm_x*pdotn*pdotn_p2/Sigma**2 + 2*Deltarprm_x*pdotn*pdotn_p2/Sigma - 2*pdotvr*pdotvr_p2*sin2thetaprm_x/(Sigma*sin2theta**2) + 2*pdotvr*pdotvr_p2prm_x/(Sigma*sin2theta) + 2*pdotvr_p2*pdotvrprm_x/(Sigma*sin2theta) - 2*Sigmaprm_x*pdotvr*pdotvr_p2/(Sigma**2*sin2theta) - 2*Sigma*pdotxir*pdotxir_p2*sin2thetaprm_x/(Lambt*sin2theta**2) + 2*Sigma*pdotxir*pdotxir_p2prm_x/(Lambt*sin2theta) + 2*Sigma*pdotxir_p2*pdotxirprm_x/(Lambt*sin2theta) + 2*Sigmaprm_x*pdotxir*pdotxir_p2/(Lambt*sin2theta) - 2*Lambtprm_x*Sigma*pdotxir*pdotxir_p2/(Lambt**2*sin2theta)
    Hnsradicand_p2prm_x = Q4_p2prm_x + gammappsum_p2prm_x
    betapsum_p2prm_x = omegatilde*pphi_p2prm_x/Lambt - Lambtprm_x*omegatilde*pphi_p2/Lambt**2
    HssTerm3_p2prm_x = Btilde**2*(Sdotn*exp2mu*xisq*(-Q_p2prm_x - Q_p2prm_x/(2*np.sqrt(Q)) + Q_p2*Qprm_x/(4*Q**(3/2))) + Sdotn*exp2mu*xisqprm_x*(-Q_p2 - Q_p2/(2*np.sqrt(Q))) + Sdotn*exp2muprm_x*xisq*(-Q_p2 - Q_p2/(2*np.sqrt(Q))) + Sdotn_p2*exp2mu*xisq*(-Qprm_x - Qprm_x/(2*np.sqrt(Q))) + Sdotn_p2*exp2mu*xisqprm_x*(-np.sqrt(Q) - Q) + Sdotn_p2*exp2muprm_x*xisq*(-np.sqrt(Q) - Q) + Sdotn_p2prm_x*exp2mu*xisq*(-np.sqrt(Q) - Q) + Sdotnprm_x*exp2mu*xisq*(-Q_p2 - Q_p2/(2*np.sqrt(Q))) + pdotvr*(-Jtilde*Sdotv*pdotn_p2prm_x - Jtilde*Sdotv_p2*pdotnprm_x - Jtilde*Sdotv_p2prm_x*pdotn - Jtilde*Sdotvprm_x*pdotn_p2 - Jtildeprm_x*Sdotv*pdotn_p2 - Jtildeprm_x*Sdotv_p2*pdotn + Sdotn*pdotvr_p2prm_x + Sdotn_p2*pdotvrprm_x + Sdotn_p2prm_x*pdotvr + Sdotnprm_x*pdotvr_p2) + pdotvr_p2*(-Jtilde*Sdotv*pdotnprm_x - Jtilde*Sdotvprm_x*pdotn - Jtildeprm_x*Sdotv*pdotn + Sdotn*pdotvrprm_x + Sdotnprm_x*pdotvr) + pdotvr_p2prm_x*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + pdotvrprm_x*(-Jtilde*Sdotv*pdotn_p2 - Jtilde*Sdotv_p2*pdotn + Sdotn*pdotvr_p2 + Sdotn_p2*pdotvr)) + Btilde*Btildeprm_x*(2*Sdotn*exp2mu*xisq*(-Q_p2 - Q_p2/(2*np.sqrt(Q))) + 2*Sdotn_p2*exp2mu*xisq*(-np.sqrt(Q) - Q) + 2*pdotvr*(-Jtilde*Sdotv*pdotn_p2 - Jtilde*Sdotv_p2*pdotn + Sdotn*pdotvr_p2 + Sdotn_p2*pdotvr) + 2*pdotvr_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr)) + expmu*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn_p2prm_x + Btilde*Jtilde*Sdotxi_p2*pdotnprm_x + Btilde*Jtilde*Sdotxi_p2prm_x*pdotn + Btilde*Jtilde*Sdotxiprm_x*pdotn_p2 + Btilde*Jtildeprm_x*Sdotxi*pdotn_p2 + Btilde*Jtildeprm_x*Sdotxi_p2*pdotn + Btildeprm_x*Jtilde*Sdotxi*pdotn_p2 + Btildeprm_x*Jtilde*Sdotxi_p2*pdotn - Sdotn*expmu*expnu*pdotxir_p2prm_x - Sdotn*expmu*expnuprm_x*pdotxir_p2 - Sdotn*expmuprm_x*expnu*pdotxir_p2 - Sdotn_p2*expmu*expnu*pdotxirprm_x - Sdotn_p2*expmu*expnuprm_x*pdotxir - Sdotn_p2*expmuprm_x*expnu*pdotxir - Sdotn_p2prm_x*expmu*expnu*pdotxir - Sdotnprm_x*expmu*expnu*pdotxir_p2) + expmu*expnu*pdotxir_p2*(Btilde*Jtilde*Sdotxi*pdotnprm_x + Btilde*Jtilde*Sdotxiprm_x*pdotn + Btilde*Jtildeprm_x*Sdotxi*pdotn + Btildeprm_x*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxirprm_x - Sdotn*expmu*expnuprm_x*pdotxir - Sdotn*expmuprm_x*expnu*pdotxir - Sdotnprm_x*expmu*expnu*pdotxir) + expmu*expnu*pdotxir_p2prm_x*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir) + expmu*expnu*pdotxirprm_x*(Btilde*Jtilde*Sdotxi*pdotn_p2 + Btilde*Jtilde*Sdotxi_p2*pdotn - Sdotn*expmu*expnu*pdotxir_p2 - Sdotn_p2*expmu*expnu*pdotxir) + expmu*expnuprm_x*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn_p2 + Btilde*Jtilde*Sdotxi_p2*pdotn - Sdotn*expmu*expnu*pdotxir_p2 - Sdotn_p2*expmu*expnu*pdotxir) + expmu*expnuprm_x*pdotxir_p2*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir) + expmuprm_x*expnu*pdotxir*(Btilde*Jtilde*Sdotxi*pdotn_p2 + Btilde*Jtilde*Sdotxi_p2*pdotn - Sdotn*expmu*expnu*pdotxir_p2 - Sdotn_p2*expmu*expnu*pdotxir) + expmuprm_x*expnu*pdotxir_p2*(Btilde*Jtilde*Sdotxi*pdotn - Sdotn*expmu*expnu*pdotxir)
    HssTerm3coeff_p2prm_x = omegacostheta*(-Q_p2prm_x - Q_p2prm_x/(2*np.sqrt(Q)) + Q_p2*Qprm_x/(4*Q**(3/2)))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**2) + omegacostheta*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))*(-2*Qprm_x - Qprm_x/np.sqrt(Q))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**3) + omegacosthetaprm_x*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**2) - expnuprm_x*omegacostheta*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu**2*(np.sqrt(Q) + Q)**2) - expmuprm_x*omegacostheta*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu**2*expnu*(np.sqrt(Q) + Q)**2) - exp2muprm_x*omegacostheta*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu**2*expmu*expnu*(np.sqrt(Q) + Q)**2) - Btildeprm_x*omegacostheta*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde**2*exp2mu*expmu*expnu*(np.sqrt(Q) + Q)**2)
    HssTerm2_p2prm_x = Btilde**2*xisq*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotn_p2prm_x - Jtilde*Sdotv_p2*pdotnprm_x - Jtilde*Sdotv_p2prm_x*pdotn - Jtilde*Sdotvprm_x*pdotn_p2 - Jtildeprm_x*Sdotv*pdotn_p2 - Jtildeprm_x*Sdotv_p2*pdotn + Sdotn*pdotvr_p2prm_x + Sdotn_p2*pdotvrprm_x + Sdotn_p2prm_x*pdotvr + Sdotnprm_x*pdotvr_p2) + Jtilde*pdotn_p2*(-Jtilde*Sdotv*pdotnprm_x - Jtilde*Sdotvprm_x*pdotn - Jtildeprm_x*Sdotv*pdotn + Sdotn*pdotvrprm_x + Sdotnprm_x*pdotvr) + Jtilde*pdotn_p2prm_x*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Jtilde*pdotnprm_x*(-Jtilde*Sdotv*pdotn_p2 - Jtilde*Sdotv_p2*pdotn + Sdotn*pdotvr_p2 + Sdotn_p2*pdotvr) + Jtildeprm_x*pdotn*(-Jtilde*Sdotv*pdotn_p2 - Jtilde*Sdotv_p2*pdotn + Sdotn*pdotvr_p2 + Sdotn_p2*pdotvr) + Jtildeprm_x*pdotn_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Sdotv*exp2mu*(Q_p2prm_x + Q_p2prm_x/(2*np.sqrt(Q)) - Q_p2*Qprm_x/(4*Q**(3/2))) + Sdotv*exp2muprm_x*(Q_p2 + Q_p2/(2*np.sqrt(Q))) + Sdotv_p2*exp2mu*(Qprm_x + Qprm_x/(2*np.sqrt(Q))) + Sdotv_p2*exp2muprm_x*(np.sqrt(Q) + Q) + Sdotv_p2prm_x*exp2mu*(np.sqrt(Q) + Q) + Sdotvprm_x*exp2mu*(Q_p2 + Q_p2/(2*np.sqrt(Q)))) + Btilde**2*xisqprm_x*(Jtilde*pdotn*(-Jtilde*Sdotv*pdotn_p2 - Jtilde*Sdotv_p2*pdotn + Sdotn*pdotvr_p2 + Sdotn_p2*pdotvr) + Jtilde*pdotn_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + Sdotv*exp2mu*(Q_p2 + Q_p2/(2*np.sqrt(Q))) + Sdotv_p2*exp2mu*(np.sqrt(Q) + Q)) + Btilde*Btildeprm_x*xisq*(2*Jtilde*pdotn*(-Jtilde*Sdotv*pdotn_p2 - Jtilde*Sdotv_p2*pdotn + Sdotn*pdotvr_p2 + Sdotn_p2*pdotvr) + 2*Jtilde*pdotn_p2*(-Jtilde*Sdotv*pdotn + Sdotn*pdotvr) + 2*Sdotv*exp2mu*(Q_p2 + Q_p2/(2*np.sqrt(Q))) + 2*Sdotv_p2*exp2mu*(np.sqrt(Q) + Q)) + expmu*pdotxir*(-Btilde*Sdotxi*expnu*pdotvr_p2prm_x - Btilde*Sdotxi*expnuprm_x*pdotvr_p2 - Btilde*Sdotxi_p2*expnu*pdotvrprm_x - Btilde*Sdotxi_p2*expnuprm_x*pdotvr - Btilde*Sdotxi_p2prm_x*expnu*pdotvr - Btilde*Sdotxiprm_x*expnu*pdotvr_p2 - Btildeprm_x*Sdotxi*expnu*pdotvr_p2 - Btildeprm_x*Sdotxi_p2*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir_p2prm_x + Sdotv*exp2nu*expmuprm_x*pdotxir_p2 + Sdotv*exp2nuprm_x*expmu*pdotxir_p2 + Sdotv_p2*exp2nu*expmu*pdotxirprm_x + Sdotv_p2*exp2nu*expmuprm_x*pdotxir + Sdotv_p2*exp2nuprm_x*expmu*pdotxir + Sdotv_p2prm_x*exp2nu*expmu*pdotxir + Sdotvprm_x*exp2nu*expmu*pdotxir_p2) + expmu*pdotxir_p2*(-Btilde*Sdotxi*expnu*pdotvrprm_x - Btilde*Sdotxi*expnuprm_x*pdotvr - Btilde*Sdotxiprm_x*expnu*pdotvr - Btildeprm_x*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxirprm_x + Sdotv*exp2nu*expmuprm_x*pdotxir + Sdotv*exp2nuprm_x*expmu*pdotxir + Sdotvprm_x*exp2nu*expmu*pdotxir) + expmu*pdotxir_p2prm_x*(-Btilde*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir) + expmu*pdotxirprm_x*(-Btilde*Sdotxi*expnu*pdotvr_p2 - Btilde*Sdotxi_p2*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir_p2 + Sdotv_p2*exp2nu*expmu*pdotxir) + expmuprm_x*pdotxir*(-Btilde*Sdotxi*expnu*pdotvr_p2 - Btilde*Sdotxi_p2*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir_p2 + Sdotv_p2*exp2nu*expmu*pdotxir) + expmuprm_x*pdotxir_p2*(-Btilde*Sdotxi*expnu*pdotvr + Sdotv*exp2nu*expmu*pdotxir)
    HssTerm2coeff_p2prm_x = Jtilde*omegar*(-Q_p2prm_x - Q_p2prm_x/(2*np.sqrt(Q)) + Q_p2*Qprm_x/(4*Q**(3/2)))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**2) + Jtilde*omegar*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))*(-2*Qprm_x - Qprm_x/np.sqrt(Q))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**3) + Jtilde*omegar*xisqprm_x*(Q_p2 + Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*xisq**2*(np.sqrt(Q) + Q)**2) - Jtilde*expnuprm_x*omegar*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu**2*xisq*(np.sqrt(Q) + Q)**2) - Jtilde*expmuprm_x*omegar*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu**2*expnu*xisq*(np.sqrt(Q) + Q)**2) - Jtilde*exp2muprm_x*omegar*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu**2*expmu*expnu*xisq*(np.sqrt(Q) + Q)**2) + Jtildeprm_x*omegar*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**2) - Btildeprm_x*Jtilde*omegar*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(2*Btilde**2*exp2mu*expmu*expnu*xisq*(np.sqrt(Q) + Q)**2)
    HssTerm1_p2prm_x = SdotSkerrhat_p2*omegaprm_x + SdotSkerrhat_p2prm_x*omega
    Hss_p2prm_x = HssTerm1_p2prm_x + HssTerm2*HssTerm2coeff_p2prm_x + HssTerm2_p2*HssTerm2coeffprm_x + HssTerm2_p2prm_x*HssTerm2coeff + HssTerm2coeff_p2*HssTerm2prm_x + HssTerm3*HssTerm3coeff_p2prm_x + HssTerm3_p2*HssTerm3coeffprm_x + HssTerm3_p2prm_x*HssTerm3coeff + HssTerm3coeff_p2*HssTerm3prm_x
    HsoTerm2c_p2prm_x = Brtilde*Jtilde*Sdotv*expmu*expnu*pdotxir_p2prm_x*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv*expmu*expnuprm_x*pdotxir_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv*expmuprm_x*expnu*pdotxir_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv_p2*expmu*expnu*pdotxirprm_x*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv_p2*expmu*expnuprm_x*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv_p2*expmuprm_x*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotv_p2prm_x*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Sdotvprm_x*expmu*expnu*pdotxir_p2*(np.sqrt(Q) + 1) + Brtilde*Jtilde*Q_p2*Sdotv*expmu*expnu*pdotxirprm_x/(2*np.sqrt(Q)) + Brtilde*Jtilde*Q_p2*Sdotv*expmu*expnuprm_x*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Q_p2*Sdotv*expmuprm_x*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Q_p2*Sdotvprm_x*expmu*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Q_p2prm_x*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q)) + Brtilde*Jtilde*Qprm_x*Sdotv*expmu*expnu*pdotxir_p2/(2*np.sqrt(Q)) + Brtilde*Jtilde*Qprm_x*Sdotv_p2*expmu*expnu*pdotxir/(2*np.sqrt(Q)) - Brtilde*Jtilde*Q_p2*Qprm_x*Sdotv*expmu*expnu*pdotxir/(4*Q**(3/2)) + Brtilde*Jtildeprm_x*Sdotv*expmu*expnu*pdotxir_p2*(np.sqrt(Q) + 1) + Brtilde*Jtildeprm_x*Sdotv_p2*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtilde*Jtildeprm_x*Q_p2*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q)) + Brtildeprm_x*Jtilde*Sdotv*expmu*expnu*pdotxir_p2*(np.sqrt(Q) + 1) + Brtildeprm_x*Jtilde*Sdotv_p2*expmu*expnu*pdotxir*(np.sqrt(Q) + 1) + Brtildeprm_x*Jtilde*Q_p2*Sdotv*expmu*expnu*pdotxir/(2*np.sqrt(Q))
    HsoTerm2b_p2prm_x = Btilde*expmu*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv_p2*nurprm_x + Jtilde*Sdotv_p2prm_x*nur + Jtildeprm_x*Sdotv_p2*nur - Sdotn_p2*nucostheta*xisqprm_x - Sdotn_p2*nucosthetaprm_x*xisq - Sdotn_p2prm_x*nucostheta*xisq) + Btilde*expmu*expnu*pdotxir_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nurprm_x + Jtilde*Sdotvprm_x*nur + Jtildeprm_x*Sdotv*nur - Sdotn*nucostheta*xisqprm_x - Sdotn*nucosthetaprm_x*xisq - Sdotnprm_x*nucostheta*xisq) + Btilde*expmu*expnu*pdotxir_p2prm_x*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*expmu*expnu*pdotxirprm_x*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv_p2*nur - Sdotn_p2*nucostheta*xisq) + Btilde*expmu*expnuprm_x*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv_p2*nur - Sdotn_p2*nucostheta*xisq) + Btilde*expmu*expnuprm_x*pdotxir_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*expmuprm_x*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv_p2*nur - Sdotn_p2*nucostheta*xisq) + Btilde*expmuprm_x*expnu*pdotxir_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btilde*Q_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nurprm_x + Jtilde*Sdotvprm_x*nur + Jtildeprm_x*Sdotv*nur - Sdotn*nucostheta*xisqprm_x - Sdotn*nucosthetaprm_x*xisq - Sdotnprm_x*nucostheta*xisq)/np.sqrt(Q) + Btilde*Q_p2*expmu*expnu*pdotxirprm_x*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Q_p2*expmu*expnuprm_x*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Q_p2*expmuprm_x*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Q_p2prm_x*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) + Btilde*Qprm_x*expmu*expnu*pdotxir*(Jtilde*Sdotv_p2*nur - Sdotn_p2*nucostheta*xisq)/np.sqrt(Q) + Btilde*Qprm_x*expmu*expnu*pdotxir_p2*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q) - Btilde*Q_p2*Qprm_x*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/(2*Q**(3/2)) + Btildeprm_x*expmu*expnu*pdotxir*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv_p2*nur - Sdotn_p2*nucostheta*xisq) + Btildeprm_x*expmu*expnu*pdotxir_p2*(2*np.sqrt(Q) + 1)*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq) + Btildeprm_x*Q_p2*expmu*expnu*pdotxir*(Jtilde*Sdotv*nur - Sdotn*nucostheta*xisq)/np.sqrt(Q)
    HsoTerm2a_p2prm_x = Btilde**2*Jtilde*Sdotxi*(-np.sqrt(Q)*(nur*pdotvr_p2prm_x + nurprm_x*pdotvr_p2 + pdotn_p2*xisq*(mucosthetaprm_x - nucosthetaprm_x) + pdotn_p2*xisqprm_x*(mucostheta - nucostheta) + pdotn_p2prm_x*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn_p2*xisqprm_x - mucostheta*pdotn_p2prm_x*xisq - mucosthetaprm_x*pdotn_p2*xisq + mur*pdotvr_p2prm_x*(np.sqrt(Q) + 1) + murprm_x*pdotvr_p2*(np.sqrt(Q) + 1) + Q_p2*mur*pdotvrprm_x/(2*np.sqrt(Q)) + Q_p2*murprm_x*pdotvr/(2*np.sqrt(Q)) + Q_p2*(-nur*pdotvrprm_x - nurprm_x*pdotvr + pdotn*xisq*(-mucosthetaprm_x + nucosthetaprm_x) + pdotn*xisqprm_x*(-mucostheta + nucostheta) + pdotnprm_x*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q)) + Q_p2prm_x*mur*pdotvr/(2*np.sqrt(Q)) + Q_p2prm_x*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q)) + Qprm_x*mur*pdotvr_p2/(2*np.sqrt(Q)) - Qprm_x*(nur*pdotvr_p2 + pdotn_p2*xisq*(mucostheta - nucostheta))/(2*np.sqrt(Q)) - Q_p2*Qprm_x*mur*pdotvr/(4*Q**(3/2)) - Q_p2*Qprm_x*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(4*Q**(3/2))) + Btilde**2*Jtilde*Sdotxi_p2*(np.sqrt(Q)*(-nur*pdotvrprm_x - nurprm_x*pdotvr + pdotn*xisq*(-mucosthetaprm_x + nucosthetaprm_x) + pdotn*xisqprm_x*(-mucostheta + nucostheta) + pdotnprm_x*xisq*(-mucostheta + nucostheta)) - mucostheta*pdotn*xisqprm_x - mucostheta*pdotnprm_x*xisq - mucosthetaprm_x*pdotn*xisq + mur*pdotvrprm_x*(np.sqrt(Q) + 1) + murprm_x*pdotvr*(np.sqrt(Q) + 1) + Qprm_x*mur*pdotvr/(2*np.sqrt(Q)) + Qprm_x*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtilde*Sdotxi_p2prm_x*(np.sqrt(Q)*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta)) - mucostheta*pdotn*xisq + mur*pdotvr*(np.sqrt(Q) + 1)) + Btilde**2*Jtilde*Sdotxiprm_x*(-np.sqrt(Q)*(nur*pdotvr_p2 + pdotn_p2*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn_p2*xisq + mur*pdotvr_p2*(np.sqrt(Q) + 1) + Q_p2*mur*pdotvr/(2*np.sqrt(Q)) + Q_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtildeprm_x*Sdotxi*(-np.sqrt(Q)*(nur*pdotvr_p2 + pdotn_p2*xisq*(mucostheta - nucostheta)) - mucostheta*pdotn_p2*xisq + mur*pdotvr_p2*(np.sqrt(Q) + 1) + Q_p2*mur*pdotvr/(2*np.sqrt(Q)) + Q_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/(2*np.sqrt(Q))) + Btilde**2*Jtildeprm_x*Sdotxi_p2*(np.sqrt(Q)*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta)) - mucostheta*pdotn*xisq + mur*pdotvr*(np.sqrt(Q) + 1)) + Btilde*Btildeprm_x*Jtilde*Sdotxi*(-2*np.sqrt(Q)*(nur*pdotvr_p2 + pdotn_p2*xisq*(mucostheta - nucostheta)) - 2*mucostheta*pdotn_p2*xisq + 2*mur*pdotvr_p2*(np.sqrt(Q) + 1) + Q_p2*mur*pdotvr/np.sqrt(Q) + Q_p2*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta))/np.sqrt(Q)) + Btilde*Btildeprm_x*Jtilde*Sdotxi_p2*(2*np.sqrt(Q)*(-nur*pdotvr + pdotn*xisq*(-mucostheta + nucostheta)) - 2*mucostheta*pdotn*xisq + 2*mur*pdotvr*(np.sqrt(Q) + 1))
    HsoTerm2_p2prm_x = HsoTerm2a_p2prm_x + HsoTerm2b_p2prm_x - HsoTerm2c_p2prm_x
    HsoTerm2coeff_p2prm_x = expnu*(-Q_p2prm_x - Q_p2prm_x/(2*np.sqrt(Q)) + Q_p2*Qprm_x/(4*Q**(3/2)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**2) + expnu*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))*(-2*Qprm_x - Qprm_x/np.sqrt(Q))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**3) + expnu*xisqprm_x*(Q_p2 + Q_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq**2*(np.sqrt(Q) + Q)**2) + expnuprm_x*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu*xisq*(np.sqrt(Q) + Q)**2) - exp2muprm_x*expnu*(-Q_p2 - Q_p2/(2*np.sqrt(Q)))/(Btilde**2*exp2mu**2*xisq*(np.sqrt(Q) + Q)**2) - Btildeprm_x*expnu*(-2*Q_p2 - Q_p2/np.sqrt(Q))/(Btilde**3*exp2mu*xisq*(np.sqrt(Q) + Q)**2)
    HsoTerm1_p2prm_x = SdotSkerrhat*exp2nu*pdotxir_p2*(-Btildeprm_x + expmu*expnuprm_x + expmuprm_x*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat*exp2nu*pdotxir_p2*xisqprm_x*(Btilde - expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq**2) + SdotSkerrhat*exp2nu*pdotxir_p2prm_x*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - SdotSkerrhat*exp2nu*expmuprm_x*pdotxir_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu**2*xisq) + SdotSkerrhat*exp2nuprm_x*pdotxir_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat_p2*exp2nu*pdotxir*(-Btildeprm_x + expmu*expnuprm_x + expmuprm_x*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - SdotSkerrhat_p2*exp2nu*pdotxir*xisqprm_x*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq**2) + SdotSkerrhat_p2*exp2nu*pdotxirprm_x*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - SdotSkerrhat_p2*exp2nu*expmuprm_x*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu**2*xisq) + SdotSkerrhat_p2*exp2nuprm_x*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhat_p2prm_x*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) + SdotSkerrhatprm_x*exp2nu*pdotxir_p2*(-Btilde + expmu*expnu)/(Btilde**2*np.sqrt(Q)*expmu*xisq) - Q_p2*SdotSkerrhat*exp2nu*pdotxir*(-Btildeprm_x + expmu*expnuprm_x + expmuprm_x*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) + Q_p2*SdotSkerrhat*exp2nu*pdotxir*xisqprm_x*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq**2) - Q_p2*SdotSkerrhat*exp2nu*pdotxirprm_x*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) + Q_p2*SdotSkerrhat*exp2nu*expmuprm_x*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu**2*xisq) - Q_p2*SdotSkerrhat*exp2nuprm_x*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Q_p2*SdotSkerrhatprm_x*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Q_p2prm_x*SdotSkerrhat*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Qprm_x*SdotSkerrhat*exp2nu*pdotxir_p2*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) - Qprm_x*SdotSkerrhat_p2*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(2*Btilde**2*Q**(3/2)*expmu*xisq) + Q_p2*Qprm_x*SdotSkerrhat*exp2nu*pdotxir*(-3*Btilde + 3*expmu*expnu)/(4*Btilde**2*Q**(5/2)*expmu*xisq) - Btildeprm_x*SdotSkerrhat*exp2nu*pdotxir_p2*(-2*Btilde + 2*expmu*expnu)/(Btilde**3*np.sqrt(Q)*expmu*xisq) - Btildeprm_x*SdotSkerrhat_p2*exp2nu*pdotxir*(-2*Btilde + 2*expmu*expnu)/(Btilde**3*np.sqrt(Q)*expmu*xisq) + Btildeprm_x*Q_p2*SdotSkerrhat*exp2nu*pdotxir*(-Btilde + expmu*expnu)/(Btilde**3*Q**(3/2)*expmu*xisq)
    Hso_p2prm_x = HsoTerm1_p2prm_x + HsoTerm2*HsoTerm2coeff_p2prm_x + HsoTerm2_p2*HsoTerm2coeffprm_x + HsoTerm2_p2prm_x*HsoTerm2coeff + HsoTerm2coeff_p2*HsoTerm2prm_x
    Hd_p2prm_x = Hdcoeff*Hdsum_p2prm_x + Hdcoeffprm_x*Hdsum_p2
    Hns_p2prm_x = betapsum_p2prm_x + Hnsradicand_p2*alphaprm_x/(2*np.sqrt(Hnsradicand)) + Hnsradicand_p2prm_x*alpha/(2*np.sqrt(Hnsradicand)) - Hnsradicand_p2*Hnsradicandprm_x*alpha/(4*Hnsradicand**(3/2))
    Hs_p2prm_x = Hso_p2prm_x + Hss_p2prm_x
    Heff_p2prm_x = -Hd_p2prm_x + Hns_p2prm_x + Hs_p2prm_x
    Hreal_p2prm_x = -Heff_p2*Heffprm_x*eta**2/(2*eta*(Heff - 1) + 1)**(3/2) + Heff_p2prm_x*eta/np.sqrt(2*eta*(Heff - 1) + 1)
    return np.array([Hreal_p2prm_x])

In [4]:
m1 = 23.
m2 = 10.
M = m1+m2
mu = m1*m2/M
eta = mu/M
S1 = np.array([0.01, 0.02, -0.03])
S2 = np.array([0.04, -0.05, 0.06])
S1 *= m1*m1
S2 *= m2*m2

SKerr = np.add(S1, S2)

# Normalize S_Kerr by total mass
SKerr /= (M*M)

# Compute a, which is a parameter in metric potentials of a Kerr spacetime
# See https://arxiv.org/abs/0912.3517 Equation (4.9)
asq = np.dot(SKerr,SKerr)
a = np.sqrt(asq)

EMgamma = 0.577215664901532860606512090082402431
KK, k0, k1, k2, k3, k4, k5, k5l, dSO, dSS = compute_const_coeffs(eta,EMgamma,a)


d2Hdxdp2 = compute_d2Hdxdp2(m1, m2, eta, 2.129681018601393e+01, 0.0, 0.0,
                        0.0, 2.335391115580442e-01, -4.235164736271502e-22,
                        S1[0], S1[1], S1[2],
                        S2[0], S2[1], S2[2], KK, k0, k1, dSO, dSS, 2, EMgamma)
d2Hdp2dx = compute_d2Hdp2dx(m1, m2, eta, 2.129681018601393e+01, 0.0, 0.0,
                        0.0, 2.335391115580442e-01, -4.235164736271502e-22,
                        S1[0], S1[1], S1[2],
                        S2[0], S2[1], S2[2], KK, k0, k1, dSO, dSS, 2, EMgamma)

#print("d2Hdxdpy = %.15e" % (d2Hdxdp2/eta))
#print("d2Hdpydx = %.15e" % (d2Hdp2dx/eta))
#print(-np.log10( abs(d2Hdxdp2 - d2Hdp2dx) / (0.5*abs(d2Hdxdp2 + d2Hdp2dx)) ) + 1)
#print(d2Hdp2dx/d2Hdxdp2)

print(d2Hdxdp2)
print("\n\n")
print(d2Hdp2dx.replace("prm_x","_x"))
print(d2Hdxdp2.replace(" ","") == d2Hdp2dx.replace("prm_x","_x").replace(" ",""))

## H_p2 has excellent agreement
    ## H_x disagrees by ~2 OOM dxdpy gives the LALSuite value while dpydx is wrong
    ## quadrupole term in H_x has excellent agreement
    ## Hs_x disagrees by ~2 OOM
        ## Hso_x disagrees by ~2 OOM
            ## HsoTerm1_x disagrees by x20
                ## HsoTerm1_x line strings agree 
                ## Btilde_x disagrees by x0.11 and minus sign!
                    ## Btilde_x line strings agree
                    ## Deltat_x disagrees by x0.11 and minus sign!
                        ## SOLUTION 1: There is a discrepancy due to the existence of a term "Deltatprm" in the Hamiltonian notebook
                ## expnu_x disagrees by x6 and minus sign; exp2nu_x is commensurate
                ## expmu_x has excellent agreement
                ## xisq_x is zero (why was this term retained?)
                ## pdotxir_x has excellent agreement
                ## SdotSKerrhat_x agrees to 4 SD
                ## Q_x agrees to 2 SD
            ## HsoTerm2_x disagrees by x7.98 and minus sign!
                ## HsoTerm2a_x disagrees by x0.18 and minus sign!
                ## 
            ## HsoTerm2coeff_x disagrees by x10
        ## Hss_x disagrees by minus sign! abs vals agree to 2 SD; weird.
    ## Hns_x disagrees by ~1 OOM
    ## Hd_x agrees up to 5 SD - should have better agreement?
## 


2*Deltau*r*r_x+Deltau_x*r**2



2*Deltau*r*r_x + 2*Deltau*r_x + Deltau_x*r**2 + 2*Deltau_x*r
False
